In [1]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta
import datetime as dt
import copy

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

#### Import Data

In [2]:
#Đọc name map để chuyển đỏi các tên thành dạng full
name_map = pd.read_excel("data/t2m_classification.xlsx", sheet_name='name_map').drop(columns=['group', 'order'],axis=1)
name_map_dict = name_map.set_index('code')['full_name'].to_dict()

order_map = pd.read_excel("data/t2m_classification.xlsx", sheet_name='name_map').drop(columns=['group', 'full_name'],axis=1)
order_map_dict = order_map.set_index('code')['order'].to_dict()

group_map = pd.read_excel("data/t2m_classification.xlsx", sheet_name='name_map').drop(columns=['order', 'full_name'],axis=1)
group_map_dict = group_map.set_index('code')['group'].to_dict()

In [3]:
#Đọc toàn bộ các file csv được xuất ra từ ami eod
eod_item_dict = {}
folder_path = 'D:\\t2m_folder\\ami_eod_data'
# folder_path = '../ami_eod_data'
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        key = os.path.splitext(filename)[0]
        eod_item_dict[key] = pd.read_csv(os.path.join(folder_path, filename)).sort_values('date', ascending=False).reset_index(drop=True)

for item, df in eod_item_dict.items():
    df['date'] = pd.to_datetime(df['date'].astype(str), format='%y%m%d')
    eod_item_dict[item] = df

#Tạo bảng tổng hớp tất cả các item
eod_item_df = pd.DataFrame(list(eod_item_dict.keys())).rename(columns={0:'item'})
eod_item_df['len'] = eod_item_df['item'].apply(lambda x: len(x))
eod_item_df['last_2chars'] = eod_item_df['item'].str[-2:]
eod_item_df['first_4chars'] = eod_item_df['item'].str[:4]

#Lọc ra danh sách tên các cổ phiếu và index
stock_name_df = eod_item_df[eod_item_df['len']==3].reset_index(drop=True).drop(['len','last_2chars','first_4chars'], axis=1)
index_name_df = eod_item_df[(eod_item_df['len']>3) & (eod_item_df['len']!=6) & (eod_item_df['len']<10) & 
                (eod_item_df['item']!='0001')].reset_index(drop=True).drop(['len','last_2chars','first_4chars'], axis=1)
                
eod_stock_dict = {k:v.drop(['option'], axis=1) for k,v in eod_item_dict.items() if k in stock_name_df['item'].tolist()}
eod_index_dict = {k:v.rename(columns={'option':'value'}).drop('cap', axis=1)
                for k,v in eod_item_dict.items() if k in index_name_df['item'].tolist()}

#Lọc ra danh sách tên các cổ phiếu, index giao dịch tự doanh và nước ngoài
stock_name_td_df = eod_item_df[(eod_item_df['len']==6) & (eod_item_df['last_2chars']=='TD')].reset_index(drop=True).drop(['len','last_2chars','first_4chars'], axis=1)
stock_name_nn_df = eod_item_df[(eod_item_df['len']==6) & (eod_item_df['last_2chars']=='NN')].reset_index(drop=True).drop(['len','last_2chars','first_4chars'], axis=1)
index_td_nn_df = eod_item_df[(eod_item_df['len']>=10) & (eod_item_df['first_4chars']!='VN30') & ((eod_item_df['last_2chars']=='NN') | (eod_item_df['last_2chars']=='TD'))].reset_index(drop=True).drop(['len','last_2chars','first_4chars'], axis=1)

stock_td_dict = {k:v.drop(['high','low','cap'], axis=1).rename(columns={'open':'sell_volume','close':'buy_volume','volume':'sell_value','option':'buy_value'})
                    for k,v in eod_item_dict.items() if k in stock_name_td_df['item'].tolist()}
stock_nn_dict = {k:v.drop(['high','low','cap'], axis=1).rename(columns={'open':'sell_volume','close':'buy_volume','volume':'sell_value','option':'buy_value'})
                    for k,v in eod_item_dict.items() if k in stock_name_nn_df['item'].tolist()}
index_td_nn_dict = {k:v.drop(['high','low','cap','stock'], axis=1).rename(columns={'open':'sell_volume','close':'buy_volume','volume':'sell_value','option':'buy_value'})
                    for k,v in eod_item_dict.items() if k in index_td_nn_df['item'].tolist()}

#Điều chỉnh đơn vị của các bảng NN và TD
for df in index_td_nn_dict.values():
    df['buy_volume'] = df['buy_volume']/1000
    df['sell_volume'] = -df['sell_volume']/1000
    df['buy_value'] = df['buy_value']/1000000000
    df['sell_value'] = -df['sell_value']/1000000000
    df['net_volume'] = df['buy_volume'] + df['sell_volume']
    df['net_value'] = df['buy_value'] + df['sell_value']

In [4]:
#Tạo một date_series bao gồm khoảng ngày tính toán eod
date_series = pd.DataFrame(eod_index_dict['VNINDEX']['date']).rename(columns={0:'date'})

#Tạo một time_series bao gồm khoảng ngày tính toán itd (tính thừa 1 ngày để trừ dần đi)
time_series_list = []
for day in date_series['date'].iloc[:1].tolist():
    time_series_list.extend(pd.date_range(start=f'{day} 09:00:00', end=f'{day} 11:25:00', freq='5T'))
    time_series_list.extend(pd.date_range(start=f'{day} 13:00:00', end=f'{day} 14:55:00', freq='5T'))
time_series = pd.DataFrame(time_series_list).rename(columns={0:'date'})

#Tạo 1 khung thời gian trong ngày từ 9h15 tới hết giờ
itd_series = pd.DataFrame(time_series_list[3:]).rename(columns={0:'date'})
itd_series['dummy'] = 0

In [5]:
#Đọc toàn bộ các file csv được xuất ra từ ami itd
itd_item_dict = {}
folder_path = 'D:\\t2m_folder\\ami_itd_data'
# folder_path = '../ami_itd_data'
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        key = os.path.splitext(filename)[0]
        itd_item_dict[key] = pd.read_csv(os.path.join(folder_path, filename)).sort_values('date', ascending=False).reset_index(drop=True)

#Lấy thời gian hiện tại của dữ liệu được xuất ra
current_time = pd.to_datetime(itd_item_dict['VNINDEX']['date'].iloc[0], format='%y%m%d %H%M%S')

#Điều chỉnh lại timeseries cho khớp với khung thời gian dữ liệu, bỏ đi các hàng chưa có dữ liệu
time_series = time_series.loc[time_series['date'] <= current_time].sort_values('date', ascending=False).reset_index(drop=True)

for item, df in itd_item_dict.items():
    
    df['date'] = pd.to_datetime(df['date'].astype(str), format='%y%m%d %H%M%S')

    #Fill dữ liệu vào các khoảng thời gian trống
    df = time_series.merge(df, on='date', how='left').sort_values('date', ascending=False)
    df[['open','high','low','close']] = df[['open','high','low','close']].fillna(method='bfill')
    df['volume'] = df['volume'].fillna(0)
    df['stock'] = item


    itd_item_dict[item] = df

#Tạo bảng tổng hợp tất cả các item
itd_item_df = pd.DataFrame(list(itd_item_dict.keys())).rename(columns={0:'item'})
itd_item_df['len'] = itd_item_df['item'].apply(lambda x: len(x))
itd_item_df['last_2chars'] = itd_item_df['item'].str[-2:]
itd_item_df['third_last_char'] = itd_item_df['item'].str[-3:-2]
itd_item_df['first_4chars'] = itd_item_df['item'].str[:4]

#Lọc ra danh sách tên các cổ phiếu và index
stock_name_df = itd_item_df[itd_item_df['len']==3].reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)
index_name_df = itd_item_df[(itd_item_df['len']>3) & (itd_item_df['len']!=6) & (itd_item_df['len']<10) & (itd_item_df['item']!='0001')]\
                .reset_index(drop=True).drop(['len','last_2chars','third_last_char','first_4chars'], axis=1)

itd_stock_dict = {k:v for k,v in itd_item_dict.items() if k in stock_name_df['item'].tolist()}
itd_index_dict = {k:v.rename(columns={'option':'value'})
                for k,v in itd_item_dict.items() if k in index_name_df['item'].tolist()}

In [6]:
def calculate_time_percent(time):
    start_time_am = dt.time(9, 00)
    end_time_am = dt.time(11, 30)
    start_time_pm = dt.time(13, 00)
    end_time_pm = dt.time(15, 00)

    def time_difference_in_minutes(time1, time2):
        delta1 = dt.timedelta(hours=time1.hour, minutes=time1.minute, seconds=time1.second)
        delta2 = dt.timedelta(hours=time2.hour, minutes=time2.minute, seconds=time2.second)
        diff = delta2 - delta1
        return diff.seconds // 60

    time = (time + timedelta(minutes=5)).time()
    full_time_range = time_difference_in_minutes(start_time_am, end_time_am) + time_difference_in_minutes(start_time_pm, end_time_pm)

    if time <= end_time_am:
        time_range = time_difference_in_minutes(start_time_am, time)
    elif time >= start_time_pm:
        time_range = time_difference_in_minutes(start_time_am, time) - time_difference_in_minutes(end_time_am, start_time_pm)

    return time_range/full_time_range

#Điều chỉnh lại time_series bỏ đi các hàng thời gian chưa có dữ liệu
time_series = time_series.loc[time_series['date'] <= current_time].sort_values('date', ascending=False).reset_index(drop=True)

#Tính thêm time percent
time_percent = time_series.copy()
time_percent['percent'] = time_percent['date'].apply(calculate_time_percent)
time_percent['percent'] = time_percent['percent'].apply(lambda x: x if x < 1 else 1)
current_time_percent = time_percent['percent'].iloc[0]

#### Đường trung bình

In [7]:
#Tính toán các đường trung bình và các đường MA
eod_stock_dict = {k: v.sort_values(by=['date'], ascending=True).reset_index(drop=True) for k, v in eod_stock_dict.items()}

eod_stock_dict = {
    key: df.assign(
        high5=df['high'].rolling(window=5, min_periods=1).max(),
        low5=df['low'].rolling(window=5, min_periods=1).min(),
        high20=df['high'].rolling(window=20, min_periods=1).max(),
        low20=df['low'].rolling(window=20, min_periods=1).min(),
        high60=df['high'].rolling(window=60, min_periods=1).max(),
        low60=df['low'].rolling(window=60, min_periods=1).min(),
        high120=df['high'].rolling(window=120, min_periods=1).max(),
        low120=df['low'].rolling(window=120, min_periods=1).min(),
        high240=df['high'].rolling(window=240, min_periods=1).max(),
        low240=df['low'].rolling(window=240, min_periods=1).min(),
        high480=df['high'].rolling(window=480, min_periods=1).max(),
        low480=df['low'].rolling(window=480, min_periods=1).min(),

        ma5_V=df['volume'].rolling(window=5, min_periods=1).mean().shift(1),
        ma20_V=df['volume'].rolling(window=20, min_periods=1).mean().shift(1),
        ma60_V=df['volume'].rolling(window=60, min_periods=1).mean().shift(1),
        ma120_V=df['volume'].rolling(window=120, min_periods=1).mean().shift(1),

        ma5=df['close'].rolling(window=5, min_periods=1).mean(),
        ma20=df['close'].rolling(window=20, min_periods=1).mean(),
        ma60=df['close'].rolling(window=60, min_periods=1).mean(),
        ma120=df['close'].rolling(window=120, min_periods=1).mean(),
        ma240=df['close'].rolling(window=240, min_periods=1).mean(),
        ma480=df['close'].rolling(window=480, min_periods=1).mean(),
    )
    for key, df in eod_stock_dict.items()
}

eod_stock_dict = {
    key: df.assign(
        trend_5p=(df['close'] > ((df['high5'] + df['low5'])/2).shift(1)).astype(int),
        trend_20p=(df['close'] > ((df['high20'] + df['low20'])/2).shift(1)).astype(int),
        trend_60p=(df['close'] > ((df['high60'] + df['low60'])/2).shift(1)).astype(int),
        trend_120p=(df['close'] > ((df['high120'] + df['low120'])/2).shift(1)).astype(int),
        trend_240p=(df['close'] > ((df['high240'] + df['low240'])/2).shift(1)).astype(int),
        trend_480p=(df['close'] > ((df['high480'] + df['low480'])/2).shift(1)).astype(int)
    )
    for key, df in eod_stock_dict.items()
}
eod_stock_dict = {k: v.sort_values(by=['date'], ascending=False).reset_index(drop=True) for k, v in eod_stock_dict.items()}

In [8]:
#Gán các đường trung bình và MA sang bảng dữ liệu ITD
for stock, df in itd_stock_dict.items():
    temp_data = eod_stock_dict[stock][['high5', 'low5', 'high20', 'low20', 'high60', 'low60',
                                       'high120','low120', 'high240', 'low240', 'high480', 'low480']].iloc[0]
    itd_stock_dict[stock] = df.assign(**temp_data)


itd_stock_dict = {k: v.sort_values(by=['date'], ascending=True).reset_index(drop=True) for k, v in itd_stock_dict.items()}
itd_stock_dict = {
    key: df.assign(
        trend_5p=(df['close'] > ((df['high5'] + df['low5'])/2).shift(1)).astype(int),
        trend_20p=(df['close'] > ((df['high20'] + df['low20'])/2).shift(1)).astype(int),
        trend_60p=(df['close'] > ((df['high60'] + df['low60'])/2).shift(1)).astype(int),
        trend_120p=(df['close'] > ((df['high120'] + df['low120'])/2).shift(1)).astype(int),
        trend_240p=(df['close'] > ((df['high240'] + df['low240'])/2).shift(1)).astype(int),
        trend_480p=(df['close'] > ((df['high480'] + df['low480'])/2).shift(1)).astype(int)
    )
    for key, df in itd_stock_dict.items()
}
itd_stock_dict = {k: v.sort_values(by=['date'], ascending=False).reset_index(drop=True) for k, v in itd_stock_dict.items()}

In [9]:
#Xoá các cổ phiếu chưa có giao dịch trong ngày
delete_stock = []
for stock, df in eod_stock_dict.items():
    if date_series['date'].iloc[0] != df['date'].iloc[0]:
        delete_stock.append(stock)
for stock in delete_stock:
    # itd_stock_dict.pop(stock)
    eod_stock_dict.pop(stock)

#Xoá các cổ phiếu có giá bị lỗi bằng 0
delete_stock = []
for stock, df in eod_stock_dict.items():
    if df['close'].min() == 0:
        delete_stock.append(stock)
for stock in delete_stock:
    itd_stock_dict.pop(stock)
    eod_stock_dict.pop(stock)

#Tính hệ số thanh khoản và đổi lại cap của cổ phiếu thành cap trung bình trong 20 phiên
for df in eod_stock_dict.values():
    df['liquid_ratio'] = df['volume'] / (df['ma5_V'])
    df['liquid_ratio'].iloc[0] = df['volume'].iloc[0] / ((df['ma5_V']).iloc[0]*current_time_percent)
    df['cap'] = df['cap'][::-1].rolling(window=20).mean()[::-1]

#### Phân nhóm cổ phiếu

In [10]:
stock_classification = pd.read_excel('data/t2m_classification.xlsx')
stock_classification = stock_classification[stock_classification['stock'].isin(list(eod_stock_dict.keys()))]

#Tạo ngày đầu tiên của tháng hiện tại
first_day_of_month = date_series[date_series['date'] > pd.Timestamp(df['date'].iloc[0].year, df['date'].iloc[0].month, 1)]['date'].iloc[-1]

#Tạo các mảng dữ liệu vốn hoá và giá của phiên đầu tiên hàng tháng
price_arr = []
cap_arr = []
for stock, df in eod_stock_dict.items():
    price_arr.append(df[df['date'] == first_day_of_month]['close'].item())
    cap_arr.append(df[df['date'] == first_day_of_month]['cap'].iloc[0].item())

#Tạo bảng chia nhóm vốn hoá
vonhoa_classification_df = stock_classification.copy()
vonhoa_classification_df['price'] = price_arr
vonhoa_classification_df['cap'] = cap_arr

cap_coef = sum(cap_arr)/10000
vonhoa_classification_df['marketcap_group'] = vonhoa_classification_df.apply(lambda x:
    'small' if ((x['cap']>cap_coef) & (x['cap']<10*cap_coef)) | 
               ((x['cap']>=10*cap_coef) & (x['cap']<20*cap_coef) & (x['price']<10)) 
               else
    ('mid' if ((x['cap']>=10*cap_coef) & (x['cap']<20*cap_coef) & (x['price']>=10)) | 
              ((x['cap']>=20*cap_coef) & (x['cap']<100*cap_coef))
              else
    ('large' if x['cap']>=100*cap_coef
               else 'penny'
)), axis=1)

stock_classification = pd.concat([stock_classification, vonhoa_classification_df['marketcap_group']], axis=1)

In [11]:
eod_all_stock = {}
eod_all_stock['all_stock'] = {key: value for key, value in eod_stock_dict.items()}

eod_industry_name = {}
for name in np.sort(stock_classification['industry_name'].unique()):
    eod_industry_name[name] = {key: value for key, value in eod_stock_dict.items() if key in stock_classification[stock_classification['industry_name'] == name]['stock'].to_list()}

eod_industry_perform = {}
for name in np.sort(stock_classification['industry_perform'].unique()):
    eod_industry_perform[name] = {key: value for key, value in eod_stock_dict.items() if key in stock_classification[stock_classification['industry_perform'] == name]['stock'].to_list()}

eod_marketcap_group = {}
for name in ['large', 'mid', 'small', 'penny']:
    eod_marketcap_group[name] = {key: value for key, value in eod_stock_dict.items() if key in stock_classification[stock_classification['marketcap_group'] == name]['stock'].to_list()}

In [12]:
itd_all_stock = {}
itd_all_stock['all_stock'] = {key: value for key, value in itd_stock_dict.items()}

itd_industry_name = {}
for name in np.sort(stock_classification['industry_name'].unique()):
    itd_industry_name[name] = {key: value for key, value in itd_stock_dict.items() if key in stock_classification[stock_classification['industry_name'] == name]['stock'].to_list()}

itd_industry_perform = {}
for name in np.sort(stock_classification['industry_perform'].unique()):
    itd_industry_perform[name] = {key: value for key, value in itd_stock_dict.items() if key in stock_classification[stock_classification['industry_perform'] == name]['stock'].to_list()}

itd_marketcap_group = {}
for name in ['large', 'mid', 'small', 'penny']:
    itd_marketcap_group[name] = {key: value for key, value in itd_stock_dict.items() if key in stock_classification[stock_classification['marketcap_group'] == name]['stock'].to_list()}

In [13]:
group_stock_list = ['all_stock'] + stock_classification['industry_name'].unique().tolist()\
                            + stock_classification['industry_perform'].unique().tolist()\
                            + stock_classification['marketcap_group'].unique().tolist()

In [14]:
#Tạo bảng để slicer các nhóm cổ phiếu
group_slicer_df = pd.DataFrame(group_stock_list).rename(columns={0:'name'})
group_slicer_df['order'] = group_slicer_df['name'].map(order_map_dict)
group_slicer_df['group'] = group_slicer_df['name'].map(group_map_dict)
group_slicer_df['name'] = group_slicer_df['name'].map(name_map_dict)

#### Biểu đồ cấu trúc sóng

In [15]:
def transform_ms(stock_group):
    stock_dict = copy.deepcopy(stock_group)
    temp_time_series = copy.deepcopy(date_series)

    for group_name in stock_dict.keys():

        #Tạo dict chứa các cổ phiếu thuộc cùng nhóm
        trend_dict = {'trend_5p': None, 'trend_20p': None,'trend_60p': None,'trend_120p': None,'trend_240p': None, 'trend_480p': None}
        for trend in trend_dict.keys():
            trend_dict[trend] = pd.DataFrame(temp_time_series['date'].tolist(), columns=['date'])
            for stock in stock_dict[group_name].keys():
                trend_dict[trend][stock] = stock_dict[group_name][stock][trend]
            trend_dict[trend].fillna(value=0, inplace=True)
            trend_dict[trend]['sum'] = trend_dict[trend].iloc[:, 1:len(stock_dict[group_name])+1].apply(sum, axis=1)
            trend_dict[trend]['percent'] = trend_dict[trend]['sum'] / len(stock_dict[group_name])

        #Tạo bảng dữ liệu theo ngày
        stock_dict[group_name] = pd.DataFrame(temp_time_series['date'].tolist(), columns=['date']).sort_values('date', ascending=False)
        for trend in trend_dict.keys():
            stock_dict[group_name][trend] = trend_dict[trend]['percent']

        stock_dict[group_name] = stock_dict[group_name].iloc[:60]

    return stock_dict

In [16]:
#Tính toán các biểu đồ MS cho các nhóm cổ phiếu
all_stock_ms = transform_ms(eod_all_stock)
industry_name_ms = transform_ms(eod_industry_name)
industry_perform_ms = transform_ms(eod_industry_perform)
marketcap_group_ms = transform_ms(eod_marketcap_group)

In [17]:
#Gộp tất cả biểu đồ MS vào 1 bảng
market_ms = pd.DataFrame()
for item in [all_stock_ms, industry_name_ms, industry_perform_ms, marketcap_group_ms]:
    for group, df in item.items():
        df['name'] = group
        market_ms = pd.concat([market_ms, df], axis=0)

market_ms['name'] = market_ms['name'].map(name_map_dict)

#### Điểm dòng tiền từng cổ phiếu

- Điểm dòng tiền EOD

In [18]:
eod_stock_dict = {k: v.iloc[:60].reset_index(drop=True) for k, v in eod_stock_dict.items()}
date_series = date_series.iloc[:60]

In [19]:
def score_calculation(df):
    try:
        result = (((df['close'] - df['low']) - (df['high'] - df['close'])) / (df['high'] - df['low']) *
                  abs((df['close'] - df['close_prev'])) / df['close_prev'] *
                  (df['volume']*df['close']) / (df['ma5_prev'] * df['ma5_V'])) * 100 \
                  + ((df['volume']*df['close']) / (df['ma5_prev'] * df['ma5_V']))/100
        result.fillna(0, inplace=True)
        return result
    except ZeroDivisionError:
        # return 0
        return ((df['volume']*df['close']) / (df['ma5_prev'] * df['ma5_V']))/100

In [20]:
#Tính toán các cột cần thiết để lọc danh sách cổ phiếu dòng tiền
raw_eod_score_dict = {}
for stock in eod_stock_dict.keys():
    raw_eod_score_dict[stock] = eod_stock_dict[stock]
    [['stock', 'date', 'high', 'low', 'close', 'volume','liquid_ratio','cap', 'ma5_V', 'ma20_V','ma60_V','ma120_V','ma5']]
    
    raw_eod_score_dict[stock]['ma5_prev'] = raw_eod_score_dict[stock]['ma5'].shift(-1)
    raw_eod_score_dict[stock]['close_prev'] = raw_eod_score_dict[stock]['close'].shift(-1)

    raw_eod_score_dict[stock]['raw_score'] = score_calculation(raw_eod_score_dict[stock])
    raw_eod_score_dict[stock]['raw_score'].iloc[0] = raw_eod_score_dict[stock]['raw_score'].iloc[0].item()/current_time_percent

    raw_eod_score_dict[stock]['highest_price'] = raw_eod_score_dict[stock]['close'][::-1].rolling(window=40, min_periods=1).max()[::-1]
    raw_eod_score_dict[stock]['lowest_volume20'] = raw_eod_score_dict[stock]['volume'][::-1].rolling(window=20, min_periods=1).min().shift(1)[::-1]
    raw_eod_score_dict[stock]['lowest_volume5'] = raw_eod_score_dict[stock]['volume'][::-1].rolling(window=5, min_periods=1).min().shift(1)[::-1]

#Lọc danh sách cổ phiếu dòng tiền
eod_score_dict = {stock: df[['stock', 'date', 'close','low','high', 'volume', 'liquid_ratio', 'raw_score','cap']] 
                    for stock, df in raw_eod_score_dict.items() 
                    if all([
                        df['ma5_V'][0] >= 50000,
                        df['ma20_V'][0] >= 50000,
                        df['ma60_V'][0] >= 50000,
                        df['ma120_V'][0] >= 50000,
                        df['lowest_volume20'][0] > 0,
                        df['lowest_volume5'][0] >= 50000,
                        df['close'][0] > df['highest_price'][0] * 0.382
                    ])}

stock_classification_filtered = stock_classification[stock_classification['stock'].isin(eod_score_dict.keys())].reset_index(drop=True)

In [21]:
#Tính toán các điểm dòng tiền đóng góp từ nhóm ngành và nhóm vốn hoá
def get_total_cap(df, group, eod_stock_dict):
    unique_groups = df[group].unique()
    total_cap_dict = {}
    for unique_group in unique_groups:
        total_cap = date_series.copy()
        keys = df[df[group] == unique_group]['stock'].tolist()
        for key in keys:
            total_cap[key] = eod_stock_dict[key]['cap']
        total_cap['total'] = total_cap.iloc[:, 1:].sum(axis=1)
        total_cap_dict[unique_group] = total_cap['total']
    return total_cap_dict

total_cap_industry_dict = get_total_cap(stock_classification, 'industry_name', eod_stock_dict)
total_cap_marketcap_dict = get_total_cap(stock_classification, 'marketcap_group', eod_stock_dict)

for stock in eod_score_dict.keys():
    eod_score_dict[stock]['total_cap_industry'] = total_cap_industry_dict[stock_classification[stock_classification['stock'] == stock]['industry_name'].item()]
    eod_score_dict[stock]['total_cap_marketcap'] = total_cap_marketcap_dict[stock_classification[stock_classification['stock'] == stock]['marketcap_group'].item()]

for stock in eod_score_dict.keys():
    eod_score_dict[stock]['industry_cont'] = eod_score_dict[stock].apply(lambda x: x['raw_score']*x['cap']/x['total_cap_industry'], axis=1)
    eod_score_dict[stock]['marketcap_cont'] = eod_score_dict[stock].apply(lambda x: x['raw_score']*x['cap']/x['total_cap_marketcap'], axis=1)

In [22]:
eod_industry_score_dict = {}
for nganh in stock_classification_filtered['industry_name'].unique():
    industry_score = date_series.copy()
    for stock in stock_classification_filtered[stock_classification_filtered['industry_name']==nganh]['stock'].tolist():
        industry_score[stock] = eod_score_dict[stock]['industry_cont']
    eod_industry_score_dict[nganh] = industry_score.iloc[:, 1:].sum(axis=1)

eod_marketcap_score_dict = {}
for nganh in stock_classification_filtered['marketcap_group'].unique():
    marketcap_score = date_series.copy()
    for stock in stock_classification_filtered[stock_classification_filtered['marketcap_group']==nganh]['stock'].tolist():
        marketcap_score[stock] = eod_score_dict[stock]['marketcap_cont']
    eod_marketcap_score_dict[nganh] = marketcap_score.iloc[:, 1:].sum(axis=1)

for stock in eod_score_dict.keys():
    nganh = stock_classification_filtered[stock_classification_filtered['stock']==stock]['industry_name'].item()
    marketcap = stock_classification_filtered[stock_classification_filtered['stock']==stock]['marketcap_group'].item()

    eod_score_dict[stock]['industry_score'] = eod_industry_score_dict[nganh]
    eod_score_dict[stock]['marketcap_score'] = eod_marketcap_score_dict[marketcap]
    eod_score_dict[stock]['t0_score'] = eod_score_dict[stock].apply(lambda x: x['raw_score']*0.7+x['industry_score']*0.2+x['marketcap_score']*0.1, axis=1)

    eod_score_dict[stock].sort_values('date', ascending=True, inplace=True)
    eod_score_dict[stock]['t5_score'] = eod_score_dict[stock]['t0_score'].rolling(window=5, min_periods=1).mean()
    eod_score_dict[stock].sort_values('date', ascending=False, inplace=True)

    eod_score_dict[stock]['industry_name'] = stock_classification_filtered[stock_classification_filtered['stock']==stock]['industry_name'].item()
    eod_score_dict[stock]['industry_perform'] = stock_classification_filtered[stock_classification_filtered['stock']==stock]['industry_perform'].item()
    eod_score_dict[stock]['stock_perform'] = stock_classification_filtered[stock_classification_filtered['stock'] == stock]['stock_perform'].item()
    eod_score_dict[stock]['marketcap_group'] = stock_classification_filtered[stock_classification_filtered['stock']==stock]['marketcap_group'].item()
    eod_score_dict[stock]['t2m_select'] = stock_classification_filtered[stock_classification_filtered['stock']==stock]['t2m_select'].item()

In [23]:
group_score = date_series.copy()
ranking_group = date_series.copy()

#Xếp hạng T5
for stock in eod_score_dict.keys():
    group_score[stock] = eod_score_dict[stock]['t5_score']
    group_score.fillna(0, inplace=True)
    ranking_group[stock] = 0
ranking_group = group_score.iloc[:,1:].rank(ascending=False, method='min', axis=1)

for stock, df in eod_score_dict.items():
    df['price_change'] = df['close'][::-1].pct_change()[::-1]
    df['value_change'] = df['close'][::-1].diff()[::-1]
    df['rank'] = ranking_group[stock]
    df['rank_prev'] = df['rank'].shift(-1)
    df['rank_change'] = df['rank_prev'] - df['rank']

#Xếp hạng T0
for stock in eod_score_dict.keys():
    group_score[stock] = eod_score_dict[stock]['t0_score']
    group_score.fillna(0, inplace=True)
    ranking_group[stock] = 0
ranking_group = group_score.iloc[:,1:].rank(ascending=False, method='min', axis=1)

for stock, df in eod_score_dict.items():
    df['rank_t0'] = ranking_group[stock]
    df['rank_t0_prev'] = df['rank_t0'].shift(-1)

#Check xem xếp hạng T0 nằm trong top 10% hay không
for stock, df in eod_score_dict.items():
    df['top_check'] = df['rank_t0'].apply(lambda x: 1 if x <= len(stock_classification_filtered)*0.1 else 0)
    df['top_count'] = df['top_check'][::-1].rolling(window=20).sum()[::-1]

# Tạo điểm số cuối cùng
eod_score_dict = {k: v.drop(
    columns=['raw_score', 'industry_cont', 'marketcap_cont', 'total_cap_industry', 'total_cap_marketcap',
             'industry_score', 'marketcap_score', 'rank_t0_prev', 'rank_prev', 'top_check'])
    for k, v in eod_score_dict.items()}

In [24]:
#Tạo bảng tổng hợp điểm t0 của tất cả cổ phiếu
eod_score_df = pd.DataFrame(stock_classification_filtered['stock'])

score_list = []
for stock, df in eod_score_dict.items():
    score_list.append(df.iloc[0])

eod_score_df = pd.DataFrame(score_list).sort_values('t0_score', ascending=False).reset_index(drop=True)
eod_score_df = eod_score_df.fillna('')

eod_score_df['industry_name'] = eod_score_df['industry_name'].map(name_map_dict)
eod_score_df['industry_perform'] = eod_score_df['industry_perform'].map(name_map_dict)
eod_score_df['marketcap_group'] = eod_score_df['marketcap_group'].map(name_map_dict)

- Điểm dòng tiền ITD

In [25]:
# Giả định date_series và itd_stock_dict đã được định nghĩa
hsx_itd_start = pd.Timestamp(date_series['date'].iloc[0].replace(hour=9, minute=15, second=0, microsecond=0))

# Danh sách stock từ stock_classification_filtered và danh sách HSX stocks
filtered_stocks = stock_classification_filtered['stock'].tolist()
hsx_stocks = stock_classification[stock_classification['exchange'] == 'HSX']['stock'].tolist()

# Lọc và cập nhật itd_score_dict trong một bước
itd_score_dict = {
    k: v.loc[v['date'] >= (hsx_itd_start if k in hsx_stocks else date_series['date'].iloc[0])]
    for k, v in copy.deepcopy(itd_stock_dict).items() if k in filtered_stocks
}

for stock, df in itd_score_dict.items():

    df['ma5_V'] = time_percent['percent']*(raw_eod_score_dict[stock]['ma5_V'].iloc[0])
    df['ma5_prev'] = raw_eod_score_dict[stock]['ma5_prev'].iloc[0]
    df['close_prev'] = raw_eod_score_dict[stock]['close_prev'].iloc[0]
    df['cap'] = raw_eod_score_dict[stock]['cap'].iloc[0]

    df['high'] = df['high'][::-1].cummax()[::-1]
    df['low'] = df['low'][::-1].cummin()[::-1]
    df['volume'] = df['volume'][::-1].cumsum()[::-1]
    df['liquid_ratio'] = df['volume']/df['ma5_V']

    df.loc[0, 'volume'] = raw_eod_score_dict[stock]['volume'].iloc[0]
    df.loc[0, 'close'] = raw_eod_score_dict[stock]['close'].iloc[0]
    df.loc[0, 'low'] = raw_eod_score_dict[stock]['low'].iloc[0]
    df.loc[0, 'high'] = raw_eod_score_dict[stock]['high'].iloc[0]

    df['raw_score'] = score_calculation(df)

for stock in itd_score_dict.keys():
    itd_score_dict[stock]['total_cap_industry'] = total_cap_industry_dict[stock_classification[stock_classification['stock'] == stock]['industry_name'].item()].iloc[0]
    itd_score_dict[stock]['total_cap_marketcap'] = total_cap_marketcap_dict[stock_classification[stock_classification['stock'] == stock]['marketcap_group'].item()].iloc[0]

for stock in itd_score_dict.keys():
    itd_score_dict[stock]['industry_cont'] = itd_score_dict[stock].apply(lambda x: x['raw_score']*x['cap']/x['total_cap_industry'], axis=1)
    itd_score_dict[stock]['marketcap_cont'] = itd_score_dict[stock].apply(lambda x: x['raw_score']*x['cap']/x['total_cap_marketcap'], axis=1)

In [26]:
itd_industry_score_dict = {}
for nganh in stock_classification_filtered['industry_name'].unique():
    industry_score = time_series.copy()
    for stock in stock_classification_filtered[stock_classification_filtered['industry_name']==nganh]['stock'].tolist():
        industry_score[stock] = itd_score_dict[stock]['industry_cont']
    itd_industry_score_dict[nganh] = industry_score.iloc[:, 1:].sum(axis=1)

itd_marketcap_score_dict = {}
for nganh in stock_classification_filtered['marketcap_group'].unique():
    marketcap_score = time_series.copy()
    for stock in stock_classification_filtered[stock_classification_filtered['marketcap_group']==nganh]['stock'].tolist():
        marketcap_score[stock] = itd_score_dict[stock]['marketcap_cont']
    itd_marketcap_score_dict[nganh] = marketcap_score.iloc[:, 1:].sum(axis=1)

for stock in itd_score_dict.keys():
    nganh = stock_classification_filtered[stock_classification_filtered['stock']==stock]['industry_name'].item()
    marketcap = stock_classification_filtered[stock_classification_filtered['stock']==stock]['marketcap_group'].item()

    itd_score_dict[stock]['industry_score'] = itd_industry_score_dict[nganh]
    itd_score_dict[stock]['marketcap_score'] = itd_marketcap_score_dict[marketcap]
    itd_score_dict[stock]['t0_score'] = itd_score_dict[stock].apply(lambda x: x['raw_score']*0.7+x['industry_score']*0.2+x['marketcap_score']*0.1, axis=1)
    itd_score_dict[stock]['price_change'] = (itd_score_dict[stock]['close'] - eod_stock_dict[stock]['open'].iloc[0])/eod_stock_dict['AAA']['open'].iloc[0]

    itd_score_dict[stock]['industry_name'] = stock_classification_filtered[stock_classification_filtered['stock']==stock]['industry_name'].item()
    itd_score_dict[stock]['industry_perform'] = stock_classification_filtered[stock_classification_filtered['stock']==stock]['industry_perform'].item()
    itd_score_dict[stock]['stock_perform'] = stock_classification_filtered[stock_classification_filtered['stock'] == stock]['stock_perform'].item()
    itd_score_dict[stock]['marketcap_group'] = stock_classification_filtered[stock_classification_filtered['stock']==stock]['marketcap_group'].item()
    itd_score_dict[stock]['t2m_select'] = stock_classification_filtered[stock_classification_filtered['stock']==stock]['t2m_select'].item()

In [27]:
itd_score_dict = {k: v[['stock', 'date','close', 'volume','t0_score', 'liquid_ratio', 'industry_name', 'industry_perform', 'stock_perform','marketcap_group', 't2m_select', 'price_change']]
                  for k, v in itd_score_dict.items()}

#### Điểm dòng tiền nhóm cổ phiếu

- Các hàm tính toán

In [28]:
#Chỉnh sửa lại điểm dòng tiền t0 cho từng cổ phiếu với tác động của độ rộng từng nhóm
def adjust_score_by_breath(t0_score, ratio_column):
    adjusted_score = []
    for score, ratio in zip(t0_score, ratio_column):
        if score >= 0:
            adjusted_score.append(score*ratio)
        else:
            adjusted_score.append(score*(1-ratio))
    return adjusted_score

#Hàm điều chỉnh điểm dòng tiền của cổ phiếu tránh sự đột biến khi đóng góp vào nhóm chung
def adjust_score_for_smooth(row, column_name, max_percent, mark):
    origin_score = row[column_name]
    
    if abs(origin_score) > row['total'] * max_percent:

        sum_abs = row['total'] - abs(row[column_name])
        fixed_score = sum_abs / (1 - max_percent) - sum_abs

        if origin_score >= 0:
            return fixed_score
        else:
            return -fixed_score
    else:
        mark[0] = 0
        return origin_score

#Áp dụng hàm điều chỉnh điểm phía trên vào các nhóm cổ phiếu, việc này lặp lại nhiều lần cho tới khi triệt tiêu sự đột biến
def apply_smooth_score(group_stock, group_name, type_name):
    if type_name == 'itd':
        score_dict = itd_score_dict
    elif type_name == 'eod':
        score_dict = eod_score_dict

    for key in group_stock.keys():

        if type_name == 'itd':
            score_df = date_series.copy()
        elif type_name == 'eod':
            score_df = time_series.copy()
        
        if group_name == 'all_stock':
            temp_stock_list = stock_classification_filtered['stock']
        else:
            temp_stock_list = stock_classification_filtered[stock_classification_filtered[f'{group_name}']==key]['stock']

        for stock in temp_stock_list:
            score_df[stock] = score_dict[stock][f't0_{group_name}']

        max_percent = max(0.1, min(5*(1/len(temp_stock_list)), 0.7))
        score_df['total'] = score_df.iloc[:, 1:].abs().sum(axis=1)

        mark = [1]
        while True:
            if mark[0] == 1:
                for stock in temp_stock_list:
                    score_df[stock] = score_df.iloc[:, 1:].apply(adjust_score_for_smooth, axis=1, args=(stock, max_percent, mark))
            if mark[0] == 0:
                break

        for stock in temp_stock_list:
            score_dict[stock][f't0_{group_name}'] = score_df[stock]

- Dòng tiền vào nhóm cổ phiếu EOD

In [29]:
#Tính độ rộng cho từng phiên phục vụ cho việc điều chỉnh điểm dòng tiền
temp_df = date_series.copy()
for stock, df in eod_score_dict.items():
    temp_df[stock] = eod_score_dict[stock]['t0_score']
temp_df.iloc[:,1:] = temp_df.iloc[:,1:].applymap(lambda x: 1 if x > 0 else 0)

eod_market_breath = date_series.copy()

industry_name_breadth_dict = {}
for key in eod_industry_name.keys():
    stock_list = stock_classification_filtered[stock_classification_filtered['industry_name']==key]['stock'].tolist()
    industry_name_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    eod_market_breath[key] = industry_name_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

industry_perform_breadth_dict = {}
for key in eod_industry_perform.keys():
    stock_list = stock_classification_filtered[stock_classification_filtered['industry_perform']==key]['stock'].tolist()
    industry_perform_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    eod_market_breath[key] = industry_perform_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

marketcap_group_breadth_dict = {}
for key in eod_marketcap_group.keys():
    stock_list = stock_classification_filtered[stock_classification_filtered['marketcap_group']==key]['stock'].tolist()
    marketcap_group_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    eod_market_breath[key] = marketcap_group_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

all_stock_breadth_dict = {}
for key in eod_all_stock.keys():
    stock_list = stock_classification_filtered['stock'].tolist()
    all_stock_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    eod_market_breath[key] = all_stock_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

#Chỉnh sửa lại điểm dòng tiền t0 cho từng cổ phiếu với tác động của độ rộng từng nhóm
for stock, df in eod_score_dict.items():
    name_of_industry_name = stock_classification_filtered[stock_classification_filtered['stock']==stock]['industry_name'].item()
    name_of_industry_perform = stock_classification_filtered[stock_classification_filtered['stock']==stock]['industry_perform'].item()
    name_of_marketcap_group = stock_classification_filtered[stock_classification_filtered['stock']==stock]['marketcap_group'].item()

    df[f't0_industry_name'] = adjust_score_by_breath(df['t0_score'], eod_market_breath[name_of_industry_name])
    df[f't0_industry_perform'] = adjust_score_by_breath(df['t0_score'], eod_market_breath[name_of_industry_perform])
    df[f't0_marketcap_group'] = adjust_score_by_breath(df['t0_score'], eod_market_breath[name_of_marketcap_group])
    df[f't0_all_stock'] = adjust_score_by_breath(df['t0_score'], eod_market_breath['all_stock'])

#Loại bỏ các giá trị điểm đột biến của các cổ phiếu khi đóng góp vào điểm dòng tiền ngành
apply_smooth_score(eod_industry_name, 'industry_name', 'eod')
apply_smooth_score(eod_industry_perform, 'industry_perform', 'eod')
apply_smooth_score(eod_marketcap_group, 'marketcap_group', 'eod')
apply_smooth_score(eod_all_stock, 'all_stock', 'eod')

In [30]:
#Tạo bảng dữ liệu điểm dòng tiền cho các nhóm cổ phiếu
eod_group_score_df = date_series.copy()

#Thêm cột điểm dòng tiền toàn bộ cổ phiếu
for nganh in eod_all_stock.keys():
    score_df = date_series.copy()
    for stock in stock_classification_filtered['stock']:
        score_df[stock] = eod_score_dict[stock]['t0_all_stock']
    score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
    eod_group_score_df[nganh] = score_df['total']

#Thêm các cột điểm dòng tiền ngành
eod_industry_name_score_df = date_series.copy()
for nganh in eod_industry_name.keys():
    score_df = date_series.copy()
    for stock in stock_classification_filtered[stock_classification_filtered['industry_name']==nganh]['stock']:
        score_df[stock] = eod_score_dict[stock]['t0_industry_name']
    score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
    eod_group_score_df[nganh] = score_df['total']

#Thêm các cột điểm dòng tiền nhóm hiệu suất
eod_industry_perform_score_df = date_series.copy()
for group in eod_industry_perform.keys():
    score_df = date_series.copy()
    for stock in stock_classification_filtered[stock_classification_filtered['industry_perform']==group]['stock']:
        score_df[stock] = eod_score_dict[stock]['t0_industry_perform']
    score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
    eod_group_score_df[group] = score_df['total']

#Thêm các cột điểm dòng tiền nhóm vốn hoá
eod_marketcap_group_score_df = date_series.copy()
for marketcap in eod_marketcap_group.keys():
    score_df = date_series.copy()
    for stock in stock_classification_filtered[stock_classification_filtered['marketcap_group']==marketcap]['stock']:
        score_df[stock] = eod_score_dict[stock]['t0_marketcap_group']
    score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
    eod_group_score_df[marketcap] = score_df['total']

eod_group_score_df['week'] = eod_group_score_df['date'].dt.strftime('%U-%Y')
eod_group_score_df['month'] = eod_group_score_df['date'].dt.strftime('%m-%Y')
eod_group_score_df['week_day'] = eod_group_score_df['date'].dt.day_name()
eod_group_score_df['day_num'] = eod_group_score_df['date'].dt.day

- Dòng tiền vào nhóm cổ phiếu ITD

In [31]:
#Tính độ rộng cho từng phiên phục vụ cho việc điều chỉnh điểm dòng tiền
temp_df = time_series.copy()
for stock, df in itd_score_dict.items():
    temp_df[stock] = itd_score_dict[stock]['t0_score']
temp_df.iloc[:,1:] = temp_df.iloc[:,1:].applymap(lambda x: 1 if x > 0 else 0)

itd_market_breath = time_series.copy()

industry_name_breadth_dict = {}
for key in itd_industry_name.keys():
    stock_list = stock_classification_filtered[stock_classification_filtered['industry_name']==key]['stock'].tolist()
    industry_name_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    itd_market_breath[key] = industry_name_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

industry_perform_breadth_dict = {}
for key in itd_industry_perform.keys():
    stock_list = stock_classification_filtered[stock_classification_filtered['industry_perform']==key]['stock'].tolist()
    industry_perform_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    itd_market_breath[key] = industry_perform_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

marketcap_group_breadth_dict = {}
for key in itd_marketcap_group.keys():
    stock_list = stock_classification_filtered[stock_classification_filtered['marketcap_group']==key]['stock'].tolist()
    marketcap_group_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    itd_market_breath[key] = marketcap_group_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

all_stock_breadth_dict = {}
for key in itd_all_stock.keys():
    stock_list = stock_classification_filtered['stock'].tolist()
    all_stock_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    itd_market_breath[key] = all_stock_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

#Chỉnh sửa lại điểm dòng tiền t0 cho từng cổ phiếu với tác động của độ rộng từng nhóm

for stock, df in itd_score_dict.items():

    name_of_industry_name = stock_classification_filtered[stock_classification_filtered['stock']==stock]['industry_name'].item()
    name_of_industry_perform = stock_classification_filtered[stock_classification_filtered['stock']==stock]['industry_perform'].item()
    name_of_marketcap_group = stock_classification_filtered[stock_classification_filtered['stock']==stock]['marketcap_group'].item()

    df['t0_industry_name'] = adjust_score_by_breath(df['t0_score'], itd_market_breath[name_of_industry_name])
    df['t0_industry_perform'] = adjust_score_by_breath(df['t0_score'], itd_market_breath[name_of_industry_perform])
    df['t0_marketcap_group'] = adjust_score_by_breath(df['t0_score'], itd_market_breath[name_of_marketcap_group])
    df['t0_all_stock'] = adjust_score_by_breath(df['t0_score'], itd_market_breath['all_stock'])

#Loại bỏ các giá trị điểm đột biến của cá cổ phiếu khi đóng góp vào điểm dòng tiền ngành
apply_smooth_score(itd_industry_name, 'industry_name', 'itd')
apply_smooth_score(itd_industry_perform, 'industry_perform', 'itd')
apply_smooth_score(itd_marketcap_group, 'marketcap_group', 'itd')
apply_smooth_score(itd_all_stock, 'all_stock', 'itd')

In [32]:
#Tạo bảng dữ liệu điểm dòng tiền cho các nhóm cổ phiếu
itd_group_score_df = time_series.copy()

#Thêm cột điểm dòng tiền toàn bộ cổ phiếu
for nganh in itd_all_stock.keys():
    score_df = time_series.copy()
    for stock in stock_classification_filtered['stock']:
        score_df[stock] = itd_score_dict[stock]['t0_all_stock']
    score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
    itd_group_score_df[nganh] = score_df['total']

#Thêm các cột điểm dòng tiền ngành
itd_industry_name_score_df = time_series.copy()
for nganh in itd_industry_name.keys():
    score_df = time_series.copy()
    for stock in stock_classification_filtered[stock_classification_filtered['industry_name']==nganh]['stock']:
        score_df[stock] = itd_score_dict[stock]['t0_industry_name']
    score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
    itd_group_score_df[nganh] = score_df['total']

#Thêm các cột điểm dòng tiền nhóm hiệu suất
itd_industry_perform_score_df = time_series.copy()
for group in itd_industry_perform.keys():
    score_df = time_series.copy()
    for stock in stock_classification_filtered[stock_classification_filtered['industry_perform']==group]['stock']:
        score_df[stock] = itd_score_dict[stock]['t0_industry_perform']
    score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
    itd_group_score_df[group] = score_df['total']

#Thêm các cột điểm dòng tiền nhóm vốn hoá
itd_marketcap_group_score_df = time_series.copy()
for marketcap in itd_marketcap_group.keys():
    score_df = time_series.copy()
    for stock in stock_classification_filtered[stock_classification_filtered['marketcap_group']==marketcap]['stock']:
        score_df[stock] = itd_score_dict[stock]['t0_marketcap_group']
    score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
    itd_group_score_df[marketcap] = score_df['total']

#### Hệ số thanh khoản cho nhóm cổ phiếu

- Hệ số thanh khoản nhóm cổ phiếu EOD

In [33]:
eod_group_liquidity_df = date_series.copy()

for name in eod_all_stock.keys():
    temp_volume_df = date_series.copy()
    for stock, df in eod_all_stock[name].items():
        temp_volume_df[stock] = df['volume']
    temp_volume_df['volume'] = temp_volume_df.iloc[:, 1:].sum(axis=1)
    temp_volume_df['ma5_V'] = temp_volume_df['volume'][::-1].shift(1).rolling(window=5).mean()[::-1]
    temp_volume_df.loc[0, 'ma5_V'] = temp_volume_df['ma5_V'].iloc[0] * current_time_percent
    temp_volume_df['ratio'] = temp_volume_df['volume'] / temp_volume_df['ma5_V']
    eod_group_liquidity_df[name] = temp_volume_df['ratio']

for name in eod_industry_name.keys():
    temp_volume_df = date_series.copy()
    for stock, df in eod_industry_name[name].items():
        temp_volume_df[stock] = df['volume']
    temp_volume_df['volume'] = temp_volume_df.iloc[:, 1:].sum(axis=1)
    temp_volume_df['ma5_V'] = temp_volume_df['volume'][::-1].shift(1).rolling(window=5).mean()[::-1]
    temp_volume_df.loc[0, 'ma5_V'] = temp_volume_df['ma5_V'].iloc[0] * current_time_percent
    temp_volume_df['ratio'] = temp_volume_df['volume'] / temp_volume_df['ma5_V']
    eod_group_liquidity_df[name] = temp_volume_df['ratio']

for name in eod_industry_perform.keys():
    temp_volume_df = date_series.copy()
    for stock, df in eod_industry_perform[name].items():
        temp_volume_df[stock] = df['volume']
    temp_volume_df['volume'] = temp_volume_df.iloc[:, 1:].sum(axis=1)
    temp_volume_df['ma5_V'] = temp_volume_df['volume'][::-1].shift(1).rolling(window=5).mean()[::-1]
    temp_volume_df.loc[0, 'ma5_V'] = temp_volume_df['ma5_V'].iloc[0] * current_time_percent
    temp_volume_df['ratio'] = temp_volume_df['volume'] / temp_volume_df['ma5_V']
    eod_group_liquidity_df[name] = temp_volume_df['ratio']

for name in eod_marketcap_group.keys():
    temp_volume_df = date_series.copy()
    for stock, df in eod_marketcap_group[name].items():
        temp_volume_df[stock] = df['volume']
    temp_volume_df['volume'] = temp_volume_df.iloc[:, 1:].sum(axis=1)
    temp_volume_df['ma5_V'] = temp_volume_df['volume'][::-1].shift(1).rolling(window=5).mean()[::-1]
    temp_volume_df.loc[0, 'ma5_V'] = temp_volume_df['ma5_V'].iloc[0] * current_time_percent
    temp_volume_df['ratio'] = temp_volume_df['volume'] / temp_volume_df['ma5_V']
    eod_group_liquidity_df[name] = temp_volume_df['ratio']

eod_group_liquidity_df = eod_group_liquidity_df.iloc[:20]

- Hệ số thanh khoản nhóm cổ phiếu ITD

In [34]:
itd_group_liquidity_df = time_series.copy().sort_values('date').reset_index(drop=True)

#Thêm cột toàn bộ cổ phiếu
for name in itd_all_stock.keys():
    liquidity_t0 = time_percent[time_percent['date'] >= date_series['date'].iloc[0]].sort_values('date').reset_index(drop=True)
    liquidity_ma5 = 0

    for stock, df in itd_all_stock[name].items():
        liquidity_t0[stock] = df[df['date'] >= date_series['date'].iloc[0]].sort_values('date')['volume'].reset_index(drop=True)
        liquidity_ma5 += eod_stock_dict[stock].iloc[0]['ma5_V']
    for column in liquidity_t0.columns[2:]:
        liquidity_t0[column] = liquidity_t0[column].cumsum()

    liquidity_t0['volume_t0'] = liquidity_t0.iloc[:,2:].sum(axis=1)
    liquidity_t0['volume_ma5'] = liquidity_ma5 * liquidity_t0['percent']
    liquidity_t0['ratio'] = liquidity_t0['volume_t0']/liquidity_t0['volume_ma5']
    liquidity_t0.loc[0, 'ratio'] = 0 

    itd_group_liquidity_df[name] = liquidity_t0['ratio']

#Thêm các cột cho các ngành
for name in itd_industry_name.keys():
    liquidity_t0 = time_percent[time_percent['date'] >= date_series['date'].iloc[0]].sort_values('date').reset_index(drop=True)
    liquidity_ma5 = 0

    for stock, df in itd_industry_name[name].items():
        liquidity_t0[stock] = df[df['date'] >= date_series['date'].iloc[0]].sort_values('date')['volume'].reset_index(drop=True)
        liquidity_ma5 += eod_stock_dict[stock].iloc[0]['ma5_V']
    for column in liquidity_t0.columns[2:]:
        liquidity_t0[column] = liquidity_t0[column].cumsum()

    liquidity_t0['volume_t0'] = liquidity_t0.iloc[:,2:].sum(axis=1)
    liquidity_t0['volume_ma5'] = liquidity_ma5 * liquidity_t0['percent']
    liquidity_t0['ratio'] = liquidity_t0['volume_t0']/liquidity_t0['volume_ma5']
    liquidity_t0.loc[0, 'ratio'] = 0 

    itd_group_liquidity_df[name] = liquidity_t0['ratio']

#Thêm các cột cho các nhóm hiệu suất
for name in itd_industry_perform.keys():
    liquidity_t0 = time_percent[time_percent['date'] >= date_series['date'].iloc[0]].sort_values('date').reset_index(drop=True)
    liquidity_ma5 = 0

    for stock, df in itd_industry_perform[name].items():
        liquidity_t0[stock] = df[df['date'] >= date_series['date'].iloc[0]].sort_values('date')['volume'].reset_index(drop=True)
        liquidity_ma5 += eod_stock_dict[stock].iloc[0]['ma5_V']
    for column in liquidity_t0.columns[2:]:
        liquidity_t0[column] = liquidity_t0[column].cumsum()

    liquidity_t0['volume_t0'] = liquidity_t0.iloc[:,2:].sum(axis=1)
    liquidity_t0['volume_ma5'] = liquidity_ma5 * liquidity_t0['percent']
    liquidity_t0['ratio'] = liquidity_t0['volume_t0']/liquidity_t0['volume_ma5']
    liquidity_t0.loc[0, 'ratio'] = 0 

    itd_group_liquidity_df[name] = liquidity_t0['ratio']

#Thêm các cột cho các nhóm vốn hoá
for name in itd_marketcap_group.keys():
    liquidity_t0 = time_percent[time_percent['date'] >= date_series['date'].iloc[0]].sort_values('date').reset_index(drop=True)
    liquidity_ma5 = 0

    for stock, df in itd_marketcap_group[name].items():
        liquidity_t0[stock] = df[df['date'] >= date_series['date'].iloc[0]].sort_values('date')['volume'].reset_index(drop=True)
        liquidity_ma5 += eod_stock_dict[stock].iloc[0]['ma5_V']
    for column in liquidity_t0.columns[2:]:
        liquidity_t0[column] = liquidity_t0[column].cumsum()

    liquidity_t0['volume_t0'] = liquidity_t0.iloc[:,2:].sum(axis=1)
    liquidity_t0['volume_ma5'] = liquidity_ma5 * liquidity_t0['percent']
    liquidity_t0['ratio'] = liquidity_t0['volume_t0']/liquidity_t0['volume_ma5']
    liquidity_t0.loc[0, 'ratio'] = 0 

    itd_group_liquidity_df[name] = liquidity_t0['ratio']

itd_group_liquidity_df = itd_group_liquidity_df.sort_values('date', ascending=False).reset_index(drop=True)

#### Gộp thanh khoản và dòng tiền các nhóm cổ phiếu

- Gộp bảng hệ số thanh khoản và dòng tiền của các nhóm cổ phiếu EOD

In [35]:
#Gộp bảng hệ số thanh khoản và dòng tiền của các nhóm cổ phiếu EOD
eod_score_liquidity_df = date_series.copy()
for column in eod_group_liquidity_df.columns[1:]:
    eod_score_liquidity_df[f'liquid_{column}'] = eod_group_liquidity_df[column]
for column in eod_group_score_df.columns[1:]:
    eod_score_liquidity_df[f'score_{column}'] = eod_group_score_df[column]

eod_score_liquidity_df = eod_score_liquidity_df.iloc[:20]

In [36]:
eod_group_liquidity_melted = eod_group_liquidity_df.iloc[:20].melt(id_vars=['date'], var_name='group_name', value_name='value')
eod_group_liquidity_melted = eod_group_liquidity_melted.rename(columns={'value':'liquidity'})
eod_group_score_melted = eod_group_score_df.iloc[:20, :-4].melt(id_vars=['date'], var_name='group_name', value_name='value')
eod_group_score_melted = eod_group_score_melted.rename(columns={'value':'score'})

eod_score_liquidity_melted = eod_group_liquidity_melted.merge(eod_group_score_melted, on=['date','group_name'], how='inner')

eod_score_liquidity_melted['group_name'] = eod_score_liquidity_melted['group_name'].map(name_map_dict)


- Gộp bảng hệ số thanh khoản và dòng tiền của các nhóm cổ phiếu ITD

In [37]:
#Gộp bảng hệ số thanh khoản và dòng tiền của các nhóm cổ phiếu ITD
itd_score_liquidity_df = time_series.copy().reset_index(drop=True)
for column in itd_group_liquidity_df.columns[1:]:
    itd_score_liquidity_df[f'liquid_{column}'] = itd_group_liquidity_df[column]
for column in itd_group_score_df.columns[1:]:
    itd_score_liquidity_df[f'score_{column}'] = itd_group_score_df[column]

itd_score_liquidity_df.iloc[0, 1:] = eod_score_liquidity_df.iloc[0, 1:-4]

#Hiệu chỉnh lại theo khung thời gian ITD
itd_score_liquidity_df = itd_series.merge(itd_score_liquidity_df, on='date', how='left')

In [38]:
#Gộp thành bảng dọc để dùng slicer
itd_group_liquidity_df = itd_series.merge(itd_group_liquidity_df, on='date', how='left').drop(columns=['dummy'], axis=1)
itd_group_liquidity_melted = itd_group_liquidity_df.melt(id_vars=['date'], var_name='group_name', value_name='value')
itd_group_liquidity_melted = itd_group_liquidity_melted.rename(columns={'value':'liquidity'})

itd_group_score_df = itd_series.merge(itd_group_score_df, on='date', how='left').drop(columns=['dummy'], axis=1)
itd_group_score_melted = itd_group_score_df.melt(id_vars=['date'], var_name='group_name', value_name='value')
itd_group_score_melted = itd_group_score_melted.rename(columns={'value':'score'})

itd_score_liquidity_melted = itd_group_liquidity_melted.merge(itd_group_score_melted, on=['date','group_name'], how='inner')
itd_score_liquidity_melted['dummy'] = 0

itd_score_liquidity_melted['group_name'] = itd_score_liquidity_melted['group_name'].map(name_map_dict)

In [39]:
#Tạo bảng giá trị cuối của dòng tiền và thanh khoản
itd_score_liquidity_last = pd.concat([eod_group_liquidity_df.iloc[0, 1:],eod_group_score_df.iloc[0,1:-4]], axis=1).reset_index()
itd_score_liquidity_last.columns = ['name','liquidity','score']

itd_score_liquidity_last['order'] = itd_score_liquidity_last['name'].map(order_map_dict)
itd_score_liquidity_last['group'] = itd_score_liquidity_last['name'].map(group_map_dict)
itd_score_liquidity_last['name'] = itd_score_liquidity_last['name'].map(name_map_dict)

#### Dòng tiền trong tuần và trong tháng

In [40]:
def fill_month_flow(series):
    new_series = series.copy()
    for i in range(len(series) - 1):
        if i == 0:
            fill_value = 0
            new_series[i] = fill_value
        else:
            fill_value = new_series[i-1]
            if pd.isna(series[i]):
                if not series[i:-1].isna().all():
                    new_series[i] = fill_value
    return new_series

- Tính toán cho từng cổ phiếu

In [41]:
stock_score_df = date_series.copy()
all_stock_list = stock_classification_filtered['stock'].tolist()

for stock, df in eod_score_dict.items():
    stock_score_df[stock] = eod_score_dict[stock]['t0_score']

stock_score_df['week'] = stock_score_df['date'].dt.strftime('%U-%Y')
stock_score_df['month'] = stock_score_df['date'].dt.strftime('%m-%Y')
stock_score_df['week_day'] = stock_score_df['date'].dt.day_name()
stock_score_df['day_num'] = stock_score_df['date'].dt.day

In [42]:
#Tạo bảng dữ liệu theo tuần
week_day_index = {'Monday': 0,'Tuesday': 1,'Wednesday': 2,'Thursday': 3,'Friday': 4,'Saturday': 5,'Sunday': 6}
week_score_dict = {}
for i in range(2):
    week_score_dict[f'week_{i+1}'] = stock_score_df[stock_score_df['week'] == stock_score_df['week'].unique()[i]].drop(columns=['date','week','month','day_num']).set_index('week_day')

    temp_df = pd.DataFrame(['Monday','Tuesday','Wednesday','Friday','Thursday']).rename(columns={0:'week_day'}).set_index('week_day')
    week_score_dict[f'week_{i+1}'] = pd.concat([temp_df, week_score_dict[f'week_{i+1}']], axis=1).reset_index()

    columns_list = week_score_dict[f'week_{i+1}'].columns
    week_score_dict[f'week_{i+1}']['id'] = f'w{i+1}'
    
    week_score_dict[f'week_{i+1}'] = week_score_dict[f'week_{i+1}'].melt(id_vars=['week_day', 'id'], value_vars=all_stock_list, var_name='stock', value_name='value')
    week_score_dict[f'week_{i+1}'] = week_score_dict[f'week_{i+1}'].pivot_table(index=['week_day', 'stock'], columns='id', values='value', aggfunc='first').reset_index()

#Bảng so sánh 2 tuần
stock_score_week = week_score_dict['week_1'].merge(week_score_dict['week_2'], on=['week_day','stock'], how='outer')
stock_score_week['day_index'] = stock_score_week['week_day'].map(week_day_index)
stock_score_week = stock_score_week.sort_values('day_index')



In [43]:
#Tạo bảng dữ liệu theo tháng
month_score_dict = {}
for i in range(2):
    month_score_dict[f'month_{i+1}'] = stock_score_df[stock_score_df['month'] == stock_score_df['month'].unique()[i]].drop(columns=['date','week','month','week_day']).set_index('day_num')

    temp_df = pd.DataFrame(list(range(0, 32))).rename(columns={0:'day_num'}).set_index('day_num')
    month_score_dict[f'month_{i+1}'] = pd.concat([temp_df, month_score_dict[f'month_{i+1}']], axis=1).reset_index()
    columns_list = month_score_dict[f'month_{i+1}'].columns


    for column in columns_list[1:]: 
        month_score_dict[f'month_{i+1}'][column] = month_score_dict[f'month_{i+1}'][column].cumsum()
        month_score_dict[f'month_{i+1}'][column].iloc[month_score_dict[f'month_{i+1}'][column].first_valid_index()-1] = 0
        month_score_dict[f'month_{i+1}'][column] = fill_month_flow(month_score_dict[f'month_{i+1}'][column])

    month_score_dict[f'month_{i+1}']['id'] = f'm{i+1}'
    
    month_score_dict[f'month_{i+1}'] = month_score_dict[f'month_{i+1}'].melt(id_vars=['day_num', 'id'], value_vars=all_stock_list, var_name='stock', value_name='value')
    month_score_dict[f'month_{i+1}'] = month_score_dict[f'month_{i+1}'].pivot_table(index=['day_num', 'stock'], columns='id', values='value', aggfunc='first').reset_index()

#Bảng so sánh các 2 tháng
stock_score_month = month_score_dict['month_1'].merge(month_score_dict['month_2'], on=['day_num','stock'], how='outer')

- Tính toán cho các nhóm cổ phiếu

In [44]:
#Tạo bảng dữ liệu theo tuần
week_day_index = {'Monday': 0,'Tuesday': 1,'Wednesday': 2,'Thursday': 3,'Friday': 4,'Saturday': 5,'Sunday': 6}
week_score_dict = {}
for i in range(2):
    week_score_dict[f'week_{i+1}'] = eod_group_score_df[eod_group_score_df['week'] == eod_group_score_df['week'].unique()[i]].drop(columns=['date','week','month','day_num']).set_index('week_day')

    temp_df = pd.DataFrame(['Monday','Tuesday','Wednesday','Friday','Thursday']).rename(columns={0:'week_day'}).set_index('week_day')
    week_score_dict[f'week_{i+1}'] = pd.concat([temp_df, week_score_dict[f'week_{i+1}']], axis=1).reset_index()

    columns_list = week_score_dict[f'week_{i+1}'].columns
    week_score_dict[f'week_{i+1}']['id'] = f'w{i+1}'
    
    week_score_dict[f'week_{i+1}'] = week_score_dict[f'week_{i+1}'].melt(id_vars=['week_day', 'id'], value_vars=group_stock_list, var_name='group_name', value_name='value')
    week_score_dict[f'week_{i+1}'] = week_score_dict[f'week_{i+1}'].pivot_table(index=['week_day', 'group_name'], columns='id', values='value', aggfunc='first').reset_index()

#Bảng so sánh 2 tuần
group_score_week = week_score_dict['week_1'].merge(week_score_dict['week_2'], on=['week_day','group_name'], how='outer')
group_score_week['day_index'] = group_score_week['week_day'].map(week_day_index)
group_score_week['group_name'] = group_score_week['group_name'].map(name_map_dict)
group_score_week = group_score_week.sort_values('day_index')

In [45]:
#Tạo bảng dữ liệu theo tháng
month_score_dict = {}
for i in range(2):
    month_score_dict[f'month_{i+1}'] = eod_group_score_df[eod_group_score_df['month'] == eod_group_score_df['month'].unique()[i]].drop(columns=['date','week','month','week_day']).set_index('day_num')

    temp_df = pd.DataFrame(list(range(0, 32))).rename(columns={0:'day_num'}).set_index('day_num')
    month_score_dict[f'month_{i+1}'] = pd.concat([temp_df, month_score_dict[f'month_{i+1}']], axis=1).reset_index()
    columns_list = month_score_dict[f'month_{i+1}'].columns


    for column in columns_list[1:]: 
        month_score_dict[f'month_{i+1}'][column] = month_score_dict[f'month_{i+1}'][column].cumsum()
        month_score_dict[f'month_{i+1}'][column].iloc[month_score_dict[f'month_{i+1}'][column].first_valid_index()-1] = 0
        month_score_dict[f'month_{i+1}'][column] = fill_month_flow(month_score_dict[f'month_{i+1}'][column])

    month_score_dict[f'month_{i+1}']['id'] = f'm{i+1}'
    
    month_score_dict[f'month_{i+1}'] = month_score_dict[f'month_{i+1}'].melt(id_vars=['day_num', 'id'], value_vars=group_stock_list, var_name='group_name', value_name='value')
    month_score_dict[f'month_{i+1}'] = month_score_dict[f'month_{i+1}'].pivot_table(index=['day_num', 'group_name'], columns='id', values='value', aggfunc='first').reset_index()

#Bảng so sánh các 2 tháng
group_score_month = month_score_dict['month_1'].merge(month_score_dict['month_2'], on=['day_num','group_name'], how='outer')
group_score_month['group_name'] = group_score_month['group_name'].map(name_map_dict)

#### Xếp hạng các nhóm cổ phiếu

In [46]:
#Tạo bảng xếp hạng cho các nhóm cổ phiếu
def create_ranking_df(score_df):
    socre_dict = {}
    for group in score_df.columns[1:]:
        socre_dict[group] = date_series.copy()
        socre_dict[group]['t0_score'] = score_df[group]
        socre_dict[group]['t5_score'] = socre_dict[group]['t0_score'][::-1].rolling(window=5).mean()[::-1]

    ranking_score = date_series.copy()
    for group in socre_dict.keys():
        ranking_score[group] = socre_dict[group]['t5_score']
        ranking_score.fillna(0,inplace=True)

    ranking_df = date_series.copy()
    for group in socre_dict.keys():
        ranking_df[group] = 0

    for i in range(len(date_series.copy())):
        ranking_df.iloc[i, 1:] = ranking_score.iloc[i, 1:].rank(ascending=False, method='min')
    
    ranking_df = ranking_df.head(20)

    return ranking_df

industry_name_ranking = create_ranking_df(eod_group_score_df[['date',                                     
        'ban_le', 'bao_hiem', 'bds', 'bds_kcn','chung_khoan', 
        'cong_nghe','cong_nghiep', 'dau_khi', 'det_may','dulich_dv',
        'dv_hatang', 'hoa_chat', 'htd', 'khoang_san', 'ngan_hang','tai_chinh',
        'thep', 'thuc_pham', 'thuy_san', 'van_tai', 'vlxd', 'xd','y_te']])
industry_perform_ranking = create_ranking_df(eod_group_score_df[['date','A', 'B', 'C', 'D']])
marketcap_group_ranking = create_ranking_df(eod_group_score_df[['date','large', 'mid', 'small', 'penny']])

group_score_ranking = industry_name_ranking.merge(industry_perform_ranking, on='date', how='left').merge(marketcap_group_ranking, on='date', how='left')

- Tạo bảng xếp hạng để sử dụng filter

In [47]:
group_score_ranking_melted = pd.DataFrame()
for column in group_score_ranking.columns[1:]:
    temp_df = group_score_ranking[['date', column]]
    temp_df.columns = [['date', 'rank']]
    temp_df['name'] = column
    group_score_ranking_melted = pd.concat([group_score_ranking_melted, temp_df], axis=0)

group_score_ranking_melted.columns = ['date','rank','name']
group_score_ranking_melted['name'] = group_score_ranking_melted['name'].map(name_map_dict)

#### Chỉ số kĩ thuật

In [48]:
# def find_peak_and_bottom(df):
#     df_copy = df.copy()

#     df_copy['close_p1'] = df_copy['close'].shift(-1)
#     df_copy['close_p2'] = df_copy['close'].shift(-2)
#     df_copy['close_f1'] = df_copy['close'].shift(1)
#     df_copy['close_f2'] = df_copy['close'].shift(2)

#     df_copy['high_p1'] = df_copy['high'].shift(-1)
#     df_copy['high_p2'] = df_copy['high'].shift(-2)
#     df_copy['high_f1'] = df_copy['high'].shift(1)
#     df_copy['high_f2'] = df_copy['high'].shift(2)

#     df_copy['low_p1'] = df_copy['low'].shift(-1)
#     df_copy['low_p2'] = df_copy['low'].shift(-2)
#     df_copy['low_f1'] = df_copy['low'].shift(1)
#     df_copy['low_f2'] = df_copy['low'].shift(2)

#     df_copy['peak'] = ((df_copy['high']>df_copy['close_p1']) & (df_copy['high']>df_copy['close_p2']) &
#                         (df_copy['high']>df_copy['close_f1']) & (df_copy['high']>df_copy['close_f2']) &
#                         (df_copy['high'] == df_copy[['high', 'high_p1', 'high_p2', 'high_f1', 'high_f2']].max(axis=1))).astype(int)

#     df_copy['bottom'] = ((df_copy['low']<df_copy['close_p1']) & (df_copy['low']<df_copy['close_p2']) &
#                         (df_copy['low']<df_copy['close_f2']) & (df_copy['low']<df_copy['close_f2']) &
#                         (df_copy['low'] == df_copy[['low', 'low_p1', 'low_p2', 'low_f1', 'low_f2']].min(axis=1))).astype(int)

#     return df_copy['peak'], df_copy['bottom']

In [49]:
# def calculate_ta_df(price_df):
#     ta_df = price_df[['stock','date','open','high','low','close','volume']]
#     ta_df['month'] = ta_df['date'].dt.to_period('M').astype(str)
#     ta_df['quarter'] = ta_df['date'].dt.to_period('Q').astype(str)
#     ta_df['year'] = ta_df['date'].dt.to_period('Y').astype(str)
#     return ta_df

def calculate_ta_df(price_df):
    ta_df = price_df[['stock', 'date', 'open', 'high', 'low', 'close', 'volume']].copy()
    ta_df['month'] = ta_df['date'].dt.to_period('M')
    ta_df['quarter'] = ta_df['date'].dt.to_period('Q')
    ta_df['year'] = ta_df['date'].dt.to_period('Y')
    return ta_df

In [50]:
def calculate_candle_ta_df(ta_df,input_type):
    ta_df_copy = ta_df.copy()
    
    ta_df_copy['month_bottom'] = ta_df_copy[ta_df_copy['month'] == ta_df_copy['month'].unique()[0]]['low'].min()
    ta_df_copy['quarter_bottom'] = ta_df_copy[ta_df_copy['quarter'] == ta_df_copy['quarter'].unique()[0]]['low'].min()
    ta_df_copy['year_bottom'] = ta_df_copy[ta_df_copy['year'] == ta_df_copy['year'].unique()[0]]['low'].min()

    ta_df_copy['month_peak'] = ta_df_copy[ta_df_copy['month'] == ta_df_copy['month'].unique()[0]]['high'].max()
    ta_df_copy['quarter_peak'] = ta_df_copy[ta_df_copy['quarter'] == ta_df_copy['quarter'].unique()[0]]['high'].max()
    ta_df_copy['year_peak'] = ta_df_copy[ta_df_copy['year'] == ta_df_copy['year'].unique()[0]]['high'].max()

    ta_df_copy['month_open'] = ta_df_copy[ta_df_copy['month'] == ta_df_copy['month'].unique()[0]]['open'].iloc[-1]
    ta_df_copy['quarter_open'] = ta_df_copy[ta_df_copy['quarter'] == ta_df_copy['quarter'].unique()[0]]['open'].iloc[-1]
    ta_df_copy['year_open'] = ta_df_copy[ta_df_copy['year'] == ta_df_copy['year'].unique()[0]]['open'].iloc[-1]

    if input_type == 'stock':

        ta_df_copy['from_month_peak'] = (ta_df_copy['close'] - ta_df_copy['month_peak'])/ta_df_copy['month_peak']
        ta_df_copy['from_quarter_peak'] = (ta_df_copy['close'] - ta_df_copy['quarter_peak'])/ta_df_copy['quarter_peak']
        ta_df_copy['from_year_peak'] = (ta_df_copy['close'] - ta_df_copy['year_peak'])/ta_df_copy['year_peak']

        ta_df_copy['from_month_bottom'] = (ta_df_copy['close'] - ta_df_copy['month_bottom'])/ta_df_copy['month_bottom']
        ta_df_copy['from_quarter_bottom'] = (ta_df_copy['close'] - ta_df_copy['quarter_bottom'])/ta_df_copy['quarter_bottom']
        ta_df_copy['from_year_bottom'] = (ta_df_copy['close'] - ta_df_copy['year_bottom'])/ta_df_copy['year_bottom']

        ta_df_copy['from_month_open'] = (ta_df_copy['close'] - ta_df_copy['month_open'])/ta_df_copy['month_open']
        ta_df_copy['from_quarter_open'] = (ta_df_copy['close'] - ta_df_copy['quarter_open'])/ta_df_copy['quarter_open']
        ta_df_copy['from_year_open'] = (ta_df_copy['close'] - ta_df_copy['year_open'])/ta_df_copy['year_open']

    if input_type == 'index':

        ta_df_copy['from_month_peak'] = (ta_df_copy['close'] - ta_df_copy['month_peak'])
        ta_df_copy['from_quarter_peak'] = (ta_df_copy['close'] - ta_df_copy['quarter_peak'])
        ta_df_copy['from_year_peak'] = (ta_df_copy['close'] - ta_df_copy['year_peak'])

        ta_df_copy['from_month_bottom'] = (ta_df_copy['close'] - ta_df_copy['month_bottom'])
        ta_df_copy['from_quarter_bottom'] = (ta_df_copy['close'] - ta_df_copy['quarter_bottom'])
        ta_df_copy['from_year_bottom'] = (ta_df_copy['close'] - ta_df_copy['year_bottom'])

        ta_df_copy['from_month_open'] = (ta_df_copy['close'] - ta_df_copy['month_open'])
        ta_df_copy['from_quarter_open'] = (ta_df_copy['close'] - ta_df_copy['quarter_open'])
        ta_df_copy['from_year_open'] = (ta_df_copy['close'] - ta_df_copy['year_open'])

    return ta_df_copy

In [51]:
def calculate_fibo_ta_df(ta_df, input_type):
    ta_df_copy = ta_df.copy()

    ta_df_copy['month_peak'] = ta_df_copy[ta_df_copy['month'].isin(ta_df_copy['month'].unique()[:2].tolist())]['high'].max()
    ta_df_copy['quarter_peak'] = ta_df_copy[ta_df_copy['quarter'].isin(ta_df_copy['quarter'].unique()[:2].tolist())]['high'].max()
    ta_df_copy['year_peak'] = ta_df_copy[ta_df_copy['year'].isin(ta_df_copy['year'].unique()[:2].tolist())]['high'].max()

    ta_df_copy['month_bottom'] = ta_df_copy[ta_df_copy['month'].isin(ta_df_copy['month'].unique()[:2].tolist())]['low'].min()
    ta_df_copy['quarter_bottom'] = ta_df_copy[ta_df_copy['quarter'].isin(ta_df_copy['quarter'].unique()[:2].tolist())]['low'].min()
    ta_df_copy['year_bottom'] = ta_df_copy[ta_df_copy['year'].isin(ta_df_copy['year'].unique()[:2].tolist())]['low'].min()
        
    ta_df_copy['month_fibo_382'] = ta_df_copy['month_peak'] - (ta_df_copy['month_peak'] - ta_df_copy['month_bottom'])*0.382
    ta_df_copy['month_fibo_500'] = ta_df_copy['month_peak'] - (ta_df_copy['month_peak'] - ta_df_copy['month_bottom'])*0.5
    ta_df_copy['month_fibo_618'] = ta_df_copy['month_peak'] - (ta_df_copy['month_peak'] - ta_df_copy['month_bottom'])*0.618

    ta_df_copy['quarter_fibo_382'] = ta_df_copy['quarter_peak'] - (ta_df_copy['quarter_peak'] - ta_df_copy['quarter_bottom'])*0.382
    ta_df_copy['quarter_fibo_500'] = ta_df_copy['quarter_peak'] - (ta_df_copy['quarter_peak'] - ta_df_copy['quarter_bottom'])*0.5
    ta_df_copy['quarter_fibo_618'] = ta_df_copy['quarter_peak'] - (ta_df_copy['quarter_peak'] - ta_df_copy['quarter_bottom'])*0.618

    ta_df_copy['year_fibo_382'] = ta_df_copy['year_peak'] - (ta_df_copy['year_peak'] - ta_df_copy['year_bottom'])*0.382
    ta_df_copy['year_fibo_500'] = ta_df_copy['year_peak'] - (ta_df_copy['year_peak'] - ta_df_copy['year_bottom'])*0.5
    ta_df_copy['year_fibo_618'] = ta_df_copy['year_peak'] - (ta_df_copy['year_peak'] - ta_df_copy['year_bottom'])*0.618

    if input_type == 'stock':

        ta_df_copy['from_month_fibo_382'] = (ta_df_copy['close'] - ta_df_copy['month_fibo_382'])/abs(ta_df_copy['month_fibo_382'])
        ta_df_copy['from_month_fibo_500'] = (ta_df_copy['close'] - ta_df_copy['month_fibo_500'])/abs(ta_df_copy['month_fibo_500'])
        ta_df_copy['from_month_fibo_618'] = (ta_df_copy['close'] - ta_df_copy['month_fibo_618'])/abs(ta_df_copy['month_fibo_618'])

        ta_df_copy['from_quarter_fibo_382'] = (ta_df_copy['close'] - ta_df_copy['quarter_fibo_382'])/abs(ta_df_copy['quarter_fibo_382'])
        ta_df_copy['from_quarter_fibo_500'] = (ta_df_copy['close'] - ta_df_copy['quarter_fibo_500'])/abs(ta_df_copy['quarter_fibo_500'])
        ta_df_copy['from_quarter_fibo_618'] = (ta_df_copy['close'] - ta_df_copy['quarter_fibo_618'])/abs(ta_df_copy['quarter_fibo_618'])

        ta_df_copy['from_year_fibo_382'] = (ta_df_copy['close'] - ta_df_copy['year_fibo_382'])/abs(ta_df_copy['year_fibo_382'])
        ta_df_copy['from_year_fibo_500'] = (ta_df_copy['close'] - ta_df_copy['year_fibo_500'])/abs(ta_df_copy['year_fibo_500'])
        ta_df_copy['from_year_fibo_618'] = (ta_df_copy['close'] - ta_df_copy['year_fibo_618'])/abs(ta_df_copy['year_fibo_618'])

    if input_type == 'index':
            
        ta_df_copy['from_month_fibo_382'] = (ta_df_copy['close'] - ta_df_copy['month_fibo_382'])
        ta_df_copy['from_month_fibo_500'] = (ta_df_copy['close'] - ta_df_copy['month_fibo_500'])
        ta_df_copy['from_month_fibo_618'] = (ta_df_copy['close'] - ta_df_copy['month_fibo_618'])

        ta_df_copy['from_quarter_fibo_382'] = (ta_df_copy['close'] - ta_df_copy['quarter_fibo_382'])
        ta_df_copy['from_quarter_fibo_500'] = (ta_df_copy['close'] - ta_df_copy['quarter_fibo_500'])
        ta_df_copy['from_quarter_fibo_618'] = (ta_df_copy['close'] - ta_df_copy['quarter_fibo_618'])

        ta_df_copy['from_year_fibo_382'] = (ta_df_copy['close'] - ta_df_copy['year_fibo_382'])
        ta_df_copy['from_year_fibo_500'] = (ta_df_copy['close'] - ta_df_copy['year_fibo_500'])
        ta_df_copy['from_year_fibo_618'] = (ta_df_copy['close'] - ta_df_copy['year_fibo_618'])

    return ta_df_copy

In [52]:
def calculate_pivot_ta_df(ta_df, input_type):
    ta_df_copy = ta_df.copy()

    try: ta_df_copy['month_peak'] = ta_df_copy[ta_df_copy['month']==ta_df_copy['month'].unique()[1]]['high'].max()
    except: ta_df_copy['month_peak'] = None
    try: ta_df_copy['quarter_peak'] = ta_df_copy[ta_df_copy['quarter']==ta_df_copy['quarter'].unique()[1]]['high'].max()
    except: ta_df_copy['quarter_peak'] = None
    try: ta_df_copy['year_peak'] = ta_df_copy[ta_df_copy['year']==ta_df_copy['year'].unique()[1]]['high'].max()
    except: ta_df_copy['year_peak'] = None

    try: ta_df_copy['month_bottom'] = ta_df_copy[ta_df_copy['month']==ta_df_copy['month'].unique()[1]]['low'].min()
    except: ta_df_copy['month_bottom'] = None
    try: ta_df_copy['quarter_bottom'] = ta_df_copy[ta_df_copy['quarter']==ta_df_copy['quarter'].unique()[1]]['low'].min()
    except: ta_df_copy['quarter_bottom'] = None
    try: ta_df_copy['year_bottom'] = ta_df_copy[ta_df_copy['year']==ta_df_copy['year'].unique()[1]]['low'].min()
    except: ta_df_copy['year_bottom'] = None

    try: ta_df_copy['month_close'] = ta_df_copy[ta_df_copy['month']==ta_df_copy['month'].unique()[1]]['close'].iloc[0]
    except: ta_df_copy['month_close'] = None
    try: ta_df_copy['quarter_close'] = ta_df_copy[ta_df_copy['quarter']==ta_df_copy['quarter'].unique()[1]]['close'].iloc[0]
    except: ta_df_copy['quarter_close'] = None
    try: ta_df_copy['year_close'] = ta_df_copy[ta_df_copy['year']==ta_df_copy['year'].unique()[1]]['close'].iloc[0]
    except: ta_df_copy['year_close'] = None

    ta_df_copy['month_pivot_p'] = (ta_df_copy['month_peak'] + ta_df_copy['month_bottom'] + ta_df_copy['month_close'])/3
    ta_df_copy['quarter_pivot_p'] = (ta_df_copy['quarter_peak'] + ta_df_copy['quarter_bottom'] + ta_df_copy['quarter_close'])/3
    ta_df_copy['year_pivot_p'] = (ta_df_copy['year_peak'] + ta_df_copy['year_bottom'] + ta_df_copy['year_close'])/3

    if input_type == 'index':
        ta_df_copy['from_month_pivot_p'] = (ta_df_copy['close'] - ta_df_copy['month_pivot_p'])
        ta_df_copy['from_quarter_pivot_p'] = (ta_df_copy['close'] - ta_df_copy['quarter_pivot_p'])
        ta_df_copy['from_year_pivot_p'] = (ta_df_copy['close'] - ta_df_copy['year_pivot_p'])

    if input_type == 'stock':
        ta_df_copy['from_month_pivot_p'] = (ta_df_copy['close'] - ta_df_copy['month_pivot_p'])/abs(ta_df_copy['month_pivot_p'])
        ta_df_copy['from_quarter_pivot_p'] = (ta_df_copy['close'] - ta_df_copy['quarter_pivot_p'])/abs(ta_df_copy['quarter_pivot_p'])
        ta_df_copy['from_year_pivot_p'] = (ta_df_copy['close'] - ta_df_copy['year_pivot_p'])/abs(ta_df_copy['year_pivot_p'])

    return ta_df_copy

In [53]:
def calculate_ma_ta_df(ta_df,input_type):
    ta_df_copy = ta_df.copy()

    ta_df_copy['ma5'] = ta_df_copy['close'][::-1].rolling(window=5, min_periods=1).mean()[::-1]
    ta_df_copy['ma20'] = ta_df_copy['close'][::-1].rolling(window=20, min_periods=1).mean()[::-1]
    ta_df_copy['ma60'] = ta_df_copy['close'][::-1].rolling(window=60, min_periods=1).mean()[::-1]
    ta_df_copy['ma120'] = ta_df_copy['close'][::-1].rolling(window=120, min_periods=1).mean()[::-1]
    ta_df_copy['ma240'] = ta_df_copy['close'][::-1].rolling(window=240, min_periods=1).mean()[::-1]
    ta_df_copy['ma480'] = ta_df_copy['close'][::-1].rolling(window=480, min_periods=1).mean()[::-1]

    if input_type == 'stock':

        ta_df_copy['from_ma5'] = (ta_df_copy['close'] - ta_df_copy['ma5'])/ta_df_copy['ma5']
        ta_df_copy['from_ma20'] = (ta_df_copy['close'] - ta_df_copy['ma20'])/ta_df_copy['ma20']
        ta_df_copy['from_ma60'] = (ta_df_copy['close'] - ta_df_copy['ma60'])/ta_df_copy['ma60']
        ta_df_copy['from_ma120'] = (ta_df_copy['close'] - ta_df_copy['ma120'])/ta_df_copy['ma120']
        ta_df_copy['from_ma240'] = (ta_df_copy['close'] - ta_df_copy['ma240'])/ta_df_copy['ma240']
        ta_df_copy['from_ma480'] = (ta_df_copy['close'] - ta_df_copy['ma480'])/ta_df_copy['ma480']

    if input_type == 'index':

        ta_df_copy['from_ma5'] = (ta_df_copy['close'] - ta_df_copy['ma5'])
        ta_df_copy['from_ma20'] = (ta_df_copy['close'] - ta_df_copy['ma20'])
        ta_df_copy['from_ma60'] = (ta_df_copy['close'] - ta_df_copy['ma60'])
        ta_df_copy['from_ma120'] = (ta_df_copy['close'] - ta_df_copy['ma120'])
        ta_df_copy['from_ma240'] = (ta_df_copy['close'] - ta_df_copy['ma240'])
        ta_df_copy['from_ma480'] = (ta_df_copy['close'] - ta_df_copy['ma480'])

    return ta_df_copy

In [161]:
def transform_ta_df(ta_df,ta_name):
    df_list = []
    for time_frame in ['month','quarter','year']:
        if ta_name == 'candle':
            df = ta_df[['stock',f'{time_frame}_open',f'{time_frame}_peak',f'{time_frame}_bottom',f'from_{time_frame}_open',f'from_{time_frame}_peak',f'from_{time_frame}_bottom']].iloc[:1]
            df_name = ['Open','High','Low']
            coef = 4
        elif ta_name == 'fibo':
            df = ta_df[['stock',f'{time_frame}_fibo_382',f'{time_frame}_fibo_500',f'{time_frame}_fibo_618',f'from_{time_frame}_fibo_382',f'from_{time_frame}_fibo_500',f'from_{time_frame}_fibo_618']].iloc[:1]
            df_name = ['Fibo 0.382', 'Fibo 0.500', 'Fibo 0.618']
            coef = 4
        elif ta_name == 'pivot':
            df = ta_df[['stock',f'{time_frame}_pivot_p',f'from_{time_frame}_pivot_p']].iloc[:1]
            df_name = ['Pivot']
            coef = 2
        elif ta_name == 'ma':
            if time_frame == 'month':
                df = ta_df[['stock','ma5','ma20','from_ma5','from_ma20']].iloc[:1]
                df_name = ['MA5','MA20']
            elif time_frame == 'quarter':
                df = ta_df[['stock','ma60','ma120','from_ma60','from_ma120']].iloc[:1]
                df_name = ['MA60','MA120']
            elif time_frame == 'year':
                df = ta_df[['stock','ma240','ma480','from_ma240','from_ma480']].iloc[:1]
                df_name = ['MA240','MA480']
            coef = 3
        df_value = df.iloc[0,1:coef].tolist()
        df_from = df.iloc[0,coef:].tolist()

        if ta_name == 'pivot':
            df_order = 3
        else:
            df_order = [i for i in range(1, len(df_name) + 1)]
            
        df = pd.DataFrame({'stock':df['stock'].item(),'name': df_name,'value': df_value,'from': df_from, 'order': df_order})
        df['id'] = time_frame
        df['ta_name'] = ta_name
        df['value'] = df['value'].apply(lambda x: '{:.2f}'.format(x) if isinstance(x, (int, float)) else x)
        df_list.append(df)
    cancat_df = pd.concat(df_list, axis=0)
    return cancat_df

def concat_ta_df(df,input_type):
    ta_df = calculate_ta_df(df)

    df_candle_raw = calculate_candle_ta_df(ta_df,input_type)
    df_pivot_raw = calculate_pivot_ta_df(ta_df,input_type)
    df_ma_raw = calculate_ma_ta_df(ta_df,input_type)
    df_fibo_raw = calculate_fibo_ta_df(ta_df,input_type)

    df_candle = transform_ta_df(df_candle_raw,'candle')
    df_pivot = transform_ta_df(df_pivot_raw,'pivot')
    df_ma = transform_ta_df(df_ma_raw,'ma')
    df_fibo = transform_ta_df(df_fibo_raw,'fibo')

    concat_ta_df = pd.concat([df_candle,df_fibo,df_pivot,df_ma], axis=0)

    ta_dict = {
        'concat_ta_df': concat_ta_df,
        'ta_dict': {
            'df_candle': df_candle_raw,
            'df_pivot': df_pivot_raw,
            'df_ma': df_ma_raw,
            'df_fibo': df_fibo_raw,
        }
    }
    return ta_dict

- Tính toán chỉ số kĩ thuật cho index

In [ ]:
ta_index_df = pd.DataFrame()
for index, df in eod_index_dict.items():
    temp_ta_dict = concat_ta_df(df, 'index')

    temp_ta_index_df = temp_ta_dict['concat_ta_df']
    ta_index_df = pd.concat([ta_index_df, temp_ta_index_df], axis=0)

- Tính toán chỉ số kĩ thuật cho cổ phiếu

In [171]:
ta_stock_df = pd.DataFrame()
ta_stock_dict = {}

for stock, df in eod_all_stock['all_stock'].items():
    df_copy = df.copy()
    temp_ta_dict = concat_ta_df(df_copy, 'stock')
    
    temp_ta_stock_df = temp_ta_dict['concat_ta_df']
    ta_stock_df = pd.concat([ta_stock_df, temp_ta_stock_df], axis=0)

    ta_stock_dict[stock] = temp_ta_dict['ta_dict']

#### Page 1: Tổng quan thị trường

- Bảng hiển thị 5 chỉ số dạng Card

In [55]:
index_card_dict = {}
for index, df in eod_index_dict.items():
    df['change_value'] = df['close'][::-1].diff()[::-1]
    df['change_percent'] = (df['close'][::-1].pct_change()[::-1]).round(4)

    index_card_dict[index] = df.iloc[0]

index_card_df = pd.DataFrame(index_card_dict).transpose().drop(['open', 'high', 'low'], axis=1).reset_index(drop=True)

- Ghép bảng vẽ biểu đồ đường cho 5 chỉ số index

In [56]:
temp_df1 = pd.DataFrame(eod_index_dict['VNINDEX']['date'])
for index, df in eod_index_dict.items():
    temp_df1[index] = df['close']

temp_df1 = temp_df1.melt(id_vars=['date'], var_name='index_name', value_name='value')

index_price_chart_df = pd.DataFrame()
for time_span, name, order in zip([60,120,240,480],['3M','6M','1Y','2Y'],[1,2,3,4]):
    for index_name in temp_df1['index_name'].unique():
        temp_df2 = temp_df1.loc[temp_df1['index_name'] == index_name].iloc[:time_span]
        temp_df2['time_span'] = name
        temp_df2['order'] = order
        index_price_chart_df = pd.concat([index_price_chart_df, temp_df2])

- Tính bảng chỉ số kĩ thuật cho các index

In [154]:
# Sử dụng dữ liệu của bảng ta_index_df

- Tâm lý và thanh khoản

In [58]:
#Tính bảng hệ số thanh khoản ITD
#Sử dụng cột all_stock trong bảng itd_score_liquidity_df

In [59]:
#Tính bảng chỉ số tâm lý
market_sentiment = time_series.copy()

for stock, df in itd_score_dict.items():
    temp_df = time_series.copy()
    temp_df['price_change'] = df['price_change'].apply(lambda x: 1 if x>= 0 else -1)
    temp_df['t0_score'] = df['t0_score'].apply(lambda x: 1 if x>= 0 else -1)
    temp_df['total'] = temp_df.iloc[:,1:].sum(axis=1)

    market_sentiment[stock] = temp_df['total'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

market_sentiment['count'] = market_sentiment.iloc[:,1:].sum(axis=1)
market_sentiment['total'] = len(itd_score_dict)

market_sentiment = market_sentiment[['date','count','total']].iloc[:-3]

market_sentiment['ratio'] = market_sentiment['count'] / market_sentiment['total']

market_sentiment['sentiment'] = market_sentiment['ratio'].apply(lambda x: 'Sợ hãi' if x < -0.8 else
                                                                ('Tiêu cực' if (x >= -0.8) & (x < -0.2) else
                                                                ('Trung lập' if (x >= -0.2) & (x < 0.2) else
                                                                ('Tích cực' if (x >= 0.2) & (x < 0.8) else 'Hưng phấn'))))

#Thêm các cột giá trị cuối để tạp card trong power bi
market_sentiment['last_ratio'] = market_sentiment['ratio'].iloc[0]
market_sentiment['last_sentiment'] = market_sentiment['sentiment'].iloc[0]

#Hiệu chỉnh lại theo khung thời gian ITD
market_sentiment = itd_series.merge(market_sentiment, on='date', how='left')

In [60]:
# #Tính bảng chỉ số tâm lý
# market_sentiment = time_series.copy()
# total_count = 0

# for stock, df in itd_score_dict.items():
#     total_count += 1
#     market_sentiment[stock] = df['t0_score']

# market_sentiment['count'] = market_sentiment.iloc[:,1:].apply(lambda row: (row > 0).sum(), axis=1)
# market_sentiment['total'] = total_count
# market_sentiment = market_sentiment[['date','count','total']].iloc[:-3]
# market_sentiment['ratio'] = market_sentiment['count'] / market_sentiment['total']
# market_sentiment['sentiment'] = market_sentiment['ratio'].apply(lambda x: 'Sợ hãi' if x < 0.1 else
#                                                                 ('Tiêu cực' if (x >= 0.1) & (x < 0.4) else
#                                                                 ('Trung lập' if (x >= 0.4) & (x < 0.6) else
#                                                                 ('Tích cực' if (x >= 0.6) & (x < 0.9) else 'Hưng phấn'))))

- Khối ngoại và tự doanh

In [61]:
#Tạo dữ liệu mua bán phiên hiện tại khối ngoại và tự doanh
def calculate_nn_td_buy_sell(index_name):
    temp_dict_nn = {}
    temp_dict_nn['KLGD_NN'] = index_td_nn_dict[f'{index_name}_NN'].iloc[0][['buy_volume','sell_volume','net_volume']].tolist()
    temp_dict_nn['GTGD_NN'] = index_td_nn_dict[f'{index_name}_NN'].iloc[0][['buy_value','sell_value','net_value']].tolist()
    nn_buy_sell_df = pd.DataFrame.from_dict(temp_dict_nn, orient='index').reset_index()
    nn_buy_sell_df.columns = ['type','Mua','Bán','Mua-Bán']
    nn_buy_sell_df = nn_buy_sell_df.set_index('type').transpose()

    temp_dict_td = {}
    temp_dict_td['KLGD_TD'] = index_td_nn_dict[f'{index_name}_TD'].iloc[0][['buy_volume','sell_volume','net_volume']].tolist()
    temp_dict_td['GTGD_TD'] = index_td_nn_dict[f'{index_name}_TD'].iloc[0][['buy_value','sell_value','net_value']].tolist()
    td_buy_sell_df = pd.DataFrame.from_dict(temp_dict_td, orient='index').reset_index()
    td_buy_sell_df.columns = ['type','Mua','Bán','Mua-Bán']
    td_buy_sell_df = td_buy_sell_df.set_index('type').transpose()

    nn_td_buy_sell_df = pd.concat([nn_buy_sell_df,td_buy_sell_df],axis=1)
    nn_td_buy_sell_df['order'] = [1,2,3]

    return nn_td_buy_sell_df

nn_td_buy_sell_hsx = calculate_nn_td_buy_sell('VNINDEX')
nn_td_buy_sell_hsx['id'] = 'HSX'
nn_td_buy_sell_hsx['order_id'] = 1
nn_td_buy_sell_hnx = calculate_nn_td_buy_sell('HNXINDEX')
nn_td_buy_sell_hnx['id'] = 'HNX'
nn_td_buy_sell_hnx['order_id'] = 2
nn_td_buy_sell_upcom = calculate_nn_td_buy_sell('UPINDEX')
nn_td_buy_sell_upcom['id'] = 'UPCOM'
nn_td_buy_sell_upcom['order_id'] = 3

nn_td_buy_sell_df = pd.concat([nn_td_buy_sell_hsx,nn_td_buy_sell_hnx,nn_td_buy_sell_upcom],axis=0).reset_index().rename(columns={'index':'type'})

#Tạo dữ liệu lịch sử 20p khối ngoại và tự doanh
nn_20p_df_hsx = index_td_nn_dict['VNINDEX_NN'][['date','net_value']].iloc[:20].rename(columns={'net_value':'nn_value'})
td_20p_df_hsx = index_td_nn_dict['VNINDEX_TD'][['date','net_value']].iloc[:20].rename(columns={'net_value':'td_value'})
nn_td_20p_df_hsx = nn_20p_df_hsx.merge(td_20p_df_hsx, how='left', on='date')
nn_td_20p_df_hsx['id'] = 'HSX'

nn_20p_df_hnx = index_td_nn_dict['HNXINDEX_NN'][['date','net_value']].iloc[:20].rename(columns={'net_value':'nn_value'})
td_20p_df_hnx = index_td_nn_dict['HNXINDEX_TD'][['date','net_value']].iloc[:20].rename(columns={'net_value':'td_value'})
nn_td_20p_df_hnx = nn_20p_df_hnx.merge(td_20p_df_hnx, how='left', on='date')
nn_td_20p_df_hnx['id'] = 'HNX'

nn_20p_df_upcom = index_td_nn_dict['UPINDEX_NN'][['date','net_value']].iloc[:20].rename(columns={'net_value':'nn_value'})
td_20p_df_upcom = index_td_nn_dict['UPINDEX_TD'][['date','net_value']].iloc[:20].rename(columns={'net_value':'td_value'})
nn_td_20p_df_upcom = nn_20p_df_upcom.merge(td_20p_df_upcom, how='left', on='date')
nn_td_20p_df_upcom['id'] = 'UPCOM'

nn_td_20p_df = pd.concat([nn_td_20p_df_hsx,nn_td_20p_df_hnx,nn_td_20p_df_upcom],axis=0)

In [62]:
def create_nn_td_top_stock(stock_dict):
    today = date_series['date'][0]
    yesterday = date_series['date'][1]
    the_day_before = date_series['date'][2]

    #Tạo ra top cổ phiếu mua bán của NN
    top_stock_dict = {}
    for stock, df in stock_dict.items():
        if not df.empty:
            if df['date'][0] == today:
                top_stock_dict[stock] = df.iloc[0,1:].tolist()
            elif df['date'][0] == yesterday:
                top_stock_dict[stock] = df.iloc[0,1:].tolist()
            elif df['date'][0] == the_day_before:
                top_stock_dict[stock] = df.iloc[0,1:].tolist()
    top_stock_df = pd.DataFrame.from_dict(top_stock_dict, orient='index').reset_index()
    top_stock_df.columns = df.columns
    top_stock_df['net_values'] = (top_stock_df['buy_value'] - top_stock_df['sell_value'])/1000000000
    top_stock_df['stock'] = top_stock_df['stock'].apply(lambda x: x[:3])

    top_sell = top_stock_df[top_stock_df['net_values']<0].sort_values('net_values')[['stock','date','net_values']].rename(columns={'stock':'sell_stock','net_values':'sell_value'}).reset_index(drop=True).head(20)
    top_buy = top_stock_df[top_stock_df['net_values']>0].sort_values('net_values', ascending=False)[['stock','net_values']].rename(columns={'stock':'buy_stock','net_values':'buy_value'}).reset_index(drop=True).head(20)
    top_stock_df = pd.concat([top_sell,top_buy], axis=1)

    return top_stock_df

try:
    nn_top_stock_hsx = create_nn_td_top_stock({k:v for k,v in stock_nn_dict.items() if k[:3] in stock_classification[stock_classification['exchange']=='HSX']['stock'].tolist()})
    nn_top_stock_hsx.columns = ['nn_sell_stock','nn_date','nn_sell_value','nn_buy_stock','nn_buy_value']
except:
    nn_top_stock_hnx = pd.DataFrame(columns=['nn_sell_stock', 'nn_date', 'nn_sell_value', 'nn_buy_stock', 'nn_buy_value'])
try:
    td_top_stock_hsx = create_nn_td_top_stock({k:v for k,v in stock_td_dict.items() if k[:3] in stock_classification[stock_classification['exchange']=='HSX']['stock'].tolist()})
    td_top_stock_hsx.columns = ['td_sell_stock','td_date','td_sell_value','td_buy_stock','td_buy_value']
except:
    td_top_stock_hsx = pd.DataFrame(columns=['td_sell_stock', 'td_date', 'td_sell_value', 'td_buy_stock', 'td_buy_value'])

nn_td_top_stock_hsx = pd.concat([nn_top_stock_hsx,td_top_stock_hsx],axis=1)
nn_td_top_stock_hsx['id'] = 'HSX'

try:
    nn_top_stock_hnx = create_nn_td_top_stock({k:v for k,v in stock_nn_dict.items() if k[:3] in stock_classification[stock_classification['exchange']=='HNX']['stock'].tolist()})
    nn_top_stock_hnx.columns = ['nn_sell_stock','nn_date','nn_sell_value','nn_buy_stock','nn_buy_value']
except:
    nn_top_stock_hnx = pd.DataFrame(columns=['nn_sell_stock', 'nn_date', 'nn_sell_value', 'nn_buy_stock', 'nn_buy_value'])

try:
    td_top_stock_hnx = create_nn_td_top_stock({k:v for k,v in stock_td_dict.items() if k[:3] in stock_classification[stock_classification['exchange']=='HNX']['stock'].tolist()})
    td_top_stock_hnx.columns = ['td_sell_stock','td_date','td_sell_value','td_buy_stock','td_buy_value']
except:
    td_top_stock_hnx = pd.DataFrame(columns=['td_sell_stock', 'td_date', 'td_sell_value', 'td_buy_stock', 'td_buy_value'])

nn_td_top_stock_hnx = pd.concat([nn_top_stock_hnx,td_top_stock_hnx],axis=1)
nn_td_top_stock_hnx['id'] = 'HNX'

try:
    nn_top_stock_upcom = create_nn_td_top_stock({k:v for k,v in stock_nn_dict.items() if k[:3] in stock_classification[stock_classification['exchange']=='UPCOM']['stock'].tolist()})
    nn_top_stock_upcom.columns = ['nn_sell_stock','nn_date','nn_sell_value','nn_buy_stock','nn_buy_value']
except:
    nn_top_stock_upcom = pd.DataFrame(columns=['td_sell_stock', 'td_date', 'td_sell_value', 'td_buy_stock', 'td_buy_value'])

try:
    td_top_stock_upcom = create_nn_td_top_stock({k:v for k,v in stock_td_dict.items() if k[:3] in stock_classification[stock_classification['exchange']=='UPCOM']['stock'].tolist()})
    td_top_stock_upcom.columns = ['td_sell_stock','td_date','td_sell_value','td_buy_stock','td_buy_value']
except:
    td_top_stock_upcom = pd.DataFrame(columns=['td_sell_stock', 'td_date', 'td_sell_value', 'td_buy_stock', 'td_buy_value'])

nn_td_top_stock_upcom = pd.concat([nn_top_stock_upcom,td_top_stock_upcom],axis=1)
nn_td_top_stock_upcom['id'] = 'UPCOM'

nn_td_top_stock = pd.concat([nn_td_top_stock_hsx,nn_td_top_stock_hnx,nn_td_top_stock_upcom],axis=0)

- Dữ liệu cho biểu đồ MS

In [63]:
#Sử dụng dữ liệu cột all_stock trong bảng market_ms

- Dữ liệu top 10 cổ phiếu tiền vào và tiền ra

In [64]:
market_top_10 = eod_score_df[['stock', 'industry_name','industry_perform','marketcap_group','close','price_change','t0_score','liquid_ratio']].sort_values('t0_score', ascending=False).iloc[:10]
market_top_10['type'] = 'top'

market_bottom_10 = eod_score_df[['stock', 'industry_name','industry_perform','marketcap_group','close','price_change','t0_score','liquid_ratio']].sort_values('t0_score', ascending=True).iloc[:10]
market_bottom_10['type'] = 'bottom'

market_top_stock = pd.concat([market_top_10, market_bottom_10], axis=0).reset_index(drop=True)

#### Page 2: Dòng tiền thị trường

- Tính toán độ rộng của tất cả nhóm cổ phiếu trong phiên T0

In [65]:
# Hàm tính độ rộng thị trường
def calculate_breadth(name, stock_list):
    up_count = 0
    down_count = 0

    for stock, df in eod_score_dict.items():
        if stock in stock_list:
            if df['t0_score'].iloc[0].item() >= 0:
                up_count += 1
            if df['t0_score'].iloc[0].item() < 0:
                down_count += 1

    return [name, up_count, down_count]

# Lấy các giá trị duy nhất từng cột và chuyển thành danh sách
industry_names = stock_classification['industry_name'].unique().tolist()
industry_performs = stock_classification['industry_perform'].unique().tolist()
marketcap_groups = stock_classification['marketcap_group'].unique().tolist()

# Gộp các danh sách lại thành một
group_stock_name_list = industry_names + industry_performs + marketcap_groups

market_breath_list = []
for name in group_stock_name_list:
    if name in industry_names:
        market_breath_list.append(calculate_breadth(name, stock_classification_filtered[stock_classification_filtered['industry_name']==name]['stock'].tolist()))
    elif name in industry_performs:
        market_breath_list.append(calculate_breadth(name, stock_classification_filtered[stock_classification_filtered['industry_perform']==name]['stock'].tolist()))
    elif name in marketcap_groups:
        market_breath_list.append(calculate_breadth(name, stock_classification_filtered[stock_classification_filtered['marketcap_group']==name]['stock'].tolist()))

market_breath_df = pd.DataFrame(market_breath_list, columns=['name','in_flow','out_flow'])
market_breath_df['group'] = market_breath_df['name'].map(group_map_dict)
market_breath_df['order'] = market_breath_df['name'].map(order_map_dict)
market_breath_df['name'] = market_breath_df['name'].map(name_map_dict)

#Thêm thứ tự theo dòng tiền của các nhóm vào bảng độ rộng
temp_df = itd_score_liquidity_last[itd_score_liquidity_last['group'].isin(['A','B','C','D'])]
temp_df = temp_df.sort_values('score', ascending=False)['name'].reset_index(drop=True).reset_index()
industry_breath_df = market_breath_df[market_breath_df['group'].isin(['A','B','C','D'])].drop(columns=['group','order'])
industry_breath_df = industry_breath_df.merge(temp_df, on='name', how='inner')

- Dữ liệu cho biểu đồ dòng tiền và thanh khoản ITD toàn thị trường

In [66]:
#Sử dụng dữ liệu của bảng itd_score_liquidity_df

- Dữ liệu cho biểu đồ dòng tiền trong tuần và trong tháng

In [67]:
#Sử dụng dữ liệu của bảng group_score_week và group_score_month

- Dữ liệu diễn biến thanh khoản toàn thị trường 20p

In [68]:
#Sử dụng dữ liệu của bảng eod_group_liquidity_df cột all_stock

- Dữ liệu cho dòng tiền và thanh khoản các nhóm cổ phiếu ITD

In [69]:
#Sử dụng dữ liệu của bảng itd_score_liquidity_df để vẽ biểu đồ ITD

In [70]:
#Sử dụng dữ liệu của bảng itd_score_liquidity_last để vẽ các biểu đồ cột

- Dữ liệu cho dòng tiền T+ và xếp hạng các nhóm trong 20 phiên

In [71]:
#Bảng diễn biến xếp hạng các nhóm cổ phiếu 20 phiên
#Sử dụng dữ liệu trong group_score_ranking

In [72]:
#Tạo bảng chỉ số dòng tiền T0 tới T5
group_score_df_5p = eod_group_score_df.iloc[:5,:-4]
group_score_df_5p['id'] = ['T-0','T-1','T-2','T-3','T-4']
group_score_df_5p = group_score_df_5p.drop(columns=['date'],axis=1).set_index('id').transpose().reset_index().rename(columns={'index':'name'})

group_score_df_5p['group'] = group_score_df_5p['name'].map(group_map_dict)
group_score_df_5p['order'] = group_score_df_5p['name'].map(order_map_dict)
group_score_df_5p['name'] = group_score_df_5p['name'].map(name_map_dict)

- Bảng xếp hạng 23 ngành theo thứ tự từ trên xuống dưới

In [73]:
full_industry_ranking = eod_group_score_df[['date',                                     
        'ban_le', 'bao_hiem', 'bds', 'bds_kcn','chung_khoan', 
        'cong_nghe','cong_nghiep', 'dau_khi', 'det_may','dulich_dv',
        'dv_hatang', 'hoa_chat', 'htd', 'khoang_san', 'ngan_hang','tai_chinh',
        'thep', 'thuc_pham', 'thuy_san', 'van_tai', 'vlxd', 'xd','y_te']]

for column in full_industry_ranking.columns[1:]:
    full_industry_ranking[column] = full_industry_ranking[column][::-1].rolling(window=5).mean()[::-1]

full_industry_ranking = pd.DataFrame(full_industry_ranking.iloc[0, 1:]).rename(columns={0: 'score'}).reset_index()
full_industry_ranking.columns = ['name', 'score']
full_industry_ranking['type'] = full_industry_ranking['score'].apply(lambda x: 'Tiền vào' if x >= 0 else 'Tiền ra')
full_industry_ranking['rank'] = full_industry_ranking['score'].rank(ascending=False, method='min')
full_industry_ranking['name'] = full_industry_ranking['name'].map(name_map_dict)

#### Page 3: Phân tích nhóm ngành

- Biểu đồ đường thể hiện index các nhóm cổ phiếu

In [74]:
def calculate_index(stock_group, name):
    price_index_date_series_copy = copy.deepcopy(price_index_date_series)

    for stock, df in stock_group[name].items():
        price_index_date_series_copy[stock] = df['close']
        price_index_date_series_copy[stock] = price_index_date_series_copy[stock][::-1].pct_change()[::-1]

    price_index_date_series_copy['total_change'] = price_index_date_series_copy.iloc[:,1:].sum(axis=1)
    price_index_date_series_copy['total_change'] = (price_index_date_series_copy['total_change']/len(stock_group[name]))*100
    price_index_date_series_copy['total_change'] = price_index_date_series_copy['total_change']*10
    price_index_date_series_copy['index_value'] = price_index_date_series_copy['total_change'][::-1].cumsum()[::-1] + 1000

    return price_index_date_series_copy['index_value']

In [75]:
#Lấy ra một date_series bao gồm năm nay và 2 năm trước
price_index_date_series = pd.DataFrame(eod_index_dict['VNINDEX']['date'])
previous_year = price_index_date_series['date'].iloc[0].year - 2
price_index_date_series = price_index_date_series.loc[price_index_date_series['date'] > pd.Timestamp(year=previous_year, month=1, day=1)]

temp_df1 = price_index_date_series.copy()

for group, df in eod_industry_name.items():
    temp_df1[group] = calculate_index(eod_industry_name, group)

for group, df in eod_industry_perform.items():
    temp_df1[group] = calculate_index(eod_industry_perform, group)

for group, df in eod_marketcap_group.items():
    temp_df1[group] = calculate_index(eod_marketcap_group, group)

#Gộp lại thành bảng dọc
temp_df1 = temp_df1.melt(id_vars=['date'], var_name='group_name', value_name='value')
temp_df1['group_name'] = temp_df1['group_name'].map(name_map_dict)

#Lặp lại thành các khung thời gian
group_stock_price_index = pd.DataFrame()
for time_span, name, order in zip([60,120,240,480],['3M','6M','1Y','2Y'],[1,2,3,4]):
    for index_name in temp_df1['group_name'].unique():
        temp_df2 = temp_df1.loc[temp_df1['group_name'] == index_name].iloc[:time_span]
        temp_df2['time_span'] = name
        temp_df2['order'] = order
        group_stock_price_index = pd.concat([group_stock_price_index, temp_df2])

- Biểu đồ MS của các nhóm cổ phiếu

In [76]:
#Sử dụng dữ liệu trong bảng market_ms

- Biểu đồ dòng tiền và thanh khoản T0

In [77]:
#Lấy từ bảng itd_score_liquidity_df

- Biểu đồ dòng tiền trong tuần và trong tháng

In [78]:
#Lấy từ bảng group_score_week và group_score_month

- Biểu đồ diễn biến xếp hạng của nhóm cổ phiếu trong 20p

In [79]:
group_score_ranking_melted = group_score_ranking.iloc[:20].melt(id_vars=['date'], var_name='group_name', value_name='value')
group_score_ranking_melted.columns = ['date', 'group_name', 'rank']
group_score_ranking_melted['group_name'] = group_score_ranking_melted['group_name'].map(name_map_dict)

- Biểu đồ hệ số thanh khoản của nhóm cổ phiếu trong 20p

In [80]:
# Sử dụng dữ liệu trong bảng eod_score_liquidity_melted

- Top cổ phiếu tích cực trong nhóm

In [85]:
group_stock_top_10_df = pd.DataFrame()

for group in group_stock_list:
    if group in stock_classification_filtered['industry_name'].unique().tolist():
        temp_group_stock_list = stock_classification_filtered[stock_classification_filtered['industry_name']==group]['stock'].tolist()
    elif group in stock_classification_filtered['industry_perform'].unique().tolist():
        temp_group_stock_list = stock_classification_filtered[stock_classification_filtered['industry_perform']==group]['stock'].tolist()
    elif group in stock_classification_filtered['marketcap_group'].unique().tolist():
        temp_group_stock_list = stock_classification_filtered[stock_classification_filtered['marketcap_group']==group]['stock'].tolist()
    else: temp_group_stock_list = []

    group_stock_df = eod_score_df[eod_score_df['stock'].isin(temp_group_stock_list)][['stock', 'industry_name','industry_perform','marketcap_group','close','price_change','t0_score','t5_score','liquid_ratio','rank']]

    group_stock_top_10 = group_stock_df.sort_values('t0_score', ascending=False).iloc[:10]
    group_stock_top_10['name'] = group
    group_stock_top_10_df = pd.concat([group_stock_top_10_df, group_stock_top_10], axis=0)

group_stock_top_10_df['name'] = group_stock_top_10_df['name'].map(name_map_dict)

#### Page 4: Phân tích cổ phiếu

- Card thông tin cổ phiếu

In [86]:
# Lấy dữ liệu từ bảng eod_score_df

- Biểu đồ giá cổ phiếu

In [127]:
temp_df1 = pd.DataFrame(eod_index_dict['VNINDEX']['date'])
for stock in stock_classification_filtered['stock'].tolist():
    temp_df1[stock] = eod_all_stock['all_stock'][stock]['close']

temp_df1 = temp_df1.melt(id_vars=['date'], var_name='stock', value_name='value')

# Pre-compute unique stocks and time spans
unique_stocks = temp_df1['stock'].unique()
time_spans = [60, 120, 240, 480]
names = ['3M', '6M', '1Y', '2Y']
orders = [1, 2, 3, 4]

# Using groupby for efficient manipulation
result_list = []
for time_span, name, order in zip(time_spans, names, orders):
    grouped = temp_df1.groupby('stock')
    for stock in unique_stocks:
        temp_df2 = grouped.get_group(stock).head(time_span)
        temp_df2['time_span'] = name
        temp_df2['order'] = order
        result_list.append(temp_df2)

# Concatenate all results outside the loop
stock_price_chart_df = pd.concat(result_list, ignore_index=True)


- Dữ liệu bảng phân tích kỹ thuật

In [ ]:
#Sử dụng dữ liệu của bảng ta_stock_df

- Biểu đồ dòng tiền và thanh khoản T0

In [139]:
stock_liquidty_score_t0 = pd.DataFrame()
for stock, df in itd_score_dict.items():
    temp_df = itd_series.merge(df, on='date', how='left')
    stock_liquidty_score_t0 = pd.concat([stock_liquidty_score_t0, temp_df[['stock', 'date','t0_score','liquid_ratio','dummy']]], axis=0)

- Biểu đồ dòng tiền trong tuần và trong tháng

In [90]:
# Lấy dữ liệu từ bảng stock_score_week và stock_score_month

- Diễn biến xếp hạng dòng tiền cổ phiếu, tương quan dòng tiền, hệ số thanh khoản

In [145]:
stock_score_power_df = pd.DataFrame()
for stock, df in eod_score_dict.items():
    temp_df = date_series.copy().iloc[:20]
    temp_df['stock'] = stock
    temp_df['close'] = eod_score_dict[stock]['close']
    temp_df['liquid_ratio'] = eod_score_dict[stock]['liquid_ratio']
    temp_df['t0_score'] = eod_score_dict[stock]['t0_score']
    temp_df['rank'] = eod_score_dict[stock]['rank']
    temp_df['rank_t0'] = eod_score_dict[stock]['rank_t0']
    temp_df['top_rank_check'] = temp_df['rank_t0'].apply(lambda x: 1 if x < temp_df['rank_t0'].max()*0.1 else 0)
    temp_df['bot_rank_check'] = temp_df['rank_t0'].apply(lambda x: 1 if x > temp_df['rank_t0'].max()*0.9 else 0)

    temp_df['score_change'] = (temp_df['t0_score'][::-1].cumsum()[::-1] - temp_df['t0_score'].iloc[-1])/100
    temp_df['price_change'] = temp_df['close'][::-1].pct_change()[::-1].fillna(0)[::-1].cumsum()[::-1]

    stock_score_power_df = pd.concat([stock_score_power_df, temp_df], axis=0)

- Diễn biến xếp hạng các nhóm ngành của cổ phiếu

In [92]:
# Lấy dữ liệu từ bảng group_score_ranking_melted

#### Page 5: Bộ lọc cổ phiếu

- Tính ra bảng cổ phiếu với đầy đủ các chỉ số kĩ thuật

In [172]:
ta_stock_df_copy = ta_stock_df.copy()
ta_stock_df_copy['full_name'] = ta_stock_df_copy['name'].apply(lambda x: x.lower().replace(' ', '_').replace('0.', ''))
ta_stock_df_copy['full_name'] = 'from'+ '_' + ta_stock_df_copy['id'] + '_' + ta_stock_df_copy['full_name'] 

ta_from_df = ta_stock_df_copy[['stock', 'full_name', 'from']]
ta_from_df = ta_from_df.pivot(index='stock', columns='full_name', values='from').reset_index()

filter_stock_df = eod_score_df.merge(ta_from_df, on='stock', how='left')

- Thêm các cột đếm các phiên trên và dưới mốc kĩ thuật

In [173]:
stock_candle_dict = {}
stock_pivot_dict = {}
stock_ma_dict = {}
stock_fibo_dict = {}

for stock in stock_classification_filtered['stock'].tolist():

    stock_candle_dict[stock] = ta_stock_dict[stock]['df_candle'][['stock',
                                'from_month_open','from_month_peak','from_month_bottom',
                                'from_quarter_open','from_quarter_peak','from_quarter_bottom',
                                'from_year_open','from_year_peak','from_year_bottom']].iloc[:10]
    stock_pivot_dict[stock] = ta_stock_dict[stock]['df_pivot'][['stock',
                                'from_month_pivot_p',
                                'from_quarter_pivot_p',
                                'from_year_pivot_p']].iloc[:10]
    stock_ma_dict[stock] = ta_stock_dict[stock]['df_ma'][['stock',
                            'from_ma5','from_ma20',
                            'from_ma60','from_ma120',
                            'from_ma240','from_ma480']].iloc[:10]
    stock_fibo_dict[stock] = ta_stock_dict[stock]['df_fibo'][['stock',
                            'from_month_fibo_382','from_month_fibo_500','from_month_fibo_618',
                            'from_quarter_fibo_382','from_quarter_fibo_500','from_quarter_fibo_618',
                            'from_year_fibo_382','from_year_fibo_500','from_year_fibo_618']].iloc[:10]

stock_ta_dict_count = {}
for stock in stock_classification_filtered['stock'].tolist():
    stock_ta_dict_count[stock] = pd.DataFrame()
    for temp_dict in [stock_candle_dict, stock_pivot_dict, stock_ma_dict, stock_fibo_dict]:
        for column in [item for item in temp_dict[stock].columns[1:] if 'trend' not in item]:
            stock_ta_dict_count[stock][f'count_{column}'] = \
            temp_dict[stock][column][::-1].rolling(window=5, min_periods=1) \
            .apply(lambda x: (x > 0).sum(), raw=True)[::-1]

stock_ta_df_count = pd.DataFrame()
for stock, df in stock_ta_dict_count.items():
    stock_ta_df_count[stock] = df.iloc[0]
stock_ta_df_count = stock_ta_df_count.transpose().reset_index().rename(columns={'index':'stock'})

filter_stock_df = pd.merge(filter_stock_df, stock_ta_df_count, on='stock', how='left')

In [177]:
filter_stock_df.columns

Index(['stock', 'date', 'close', 'low', 'high', 'volume', 'liquid_ratio',
       'cap', 't0_score', 't5_score', 'industry_name', 'industry_perform',
       'stock_perform', 'marketcap_group', 't2m_select', 'price_change',
       'value_change', 'rank', 'rank_change', 'rank_t0', 'top_count',
       'from_month_fibo_382', 'from_month_fibo_500', 'from_month_fibo_618',
       'from_month_high', 'from_month_low', 'from_month_ma20',
       'from_month_ma5', 'from_month_open', 'from_month_pivot',
       'from_quarter_fibo_382', 'from_quarter_fibo_500',
       'from_quarter_fibo_618', 'from_quarter_high', 'from_quarter_low',
       'from_quarter_ma120', 'from_quarter_ma60', 'from_quarter_open',
       'from_quarter_pivot', 'from_year_fibo_382', 'from_year_fibo_500',
       'from_year_fibo_618', 'from_year_high', 'from_year_low',
       'from_year_ma240', 'from_year_ma480', 'from_year_open',
       'from_year_pivot', 'count_from_month_open', 'count_from_month_peak',
       'count_from_month_bot

- Ghép các cột lại là tính ra kháng cự hỗ trợ

In [176]:
temp_list = ['_month_open', '_month_high', '_month_low','_quarter_open', '_quarter_high', '_quarter_low',
            '_year_open', '_year_high', '_year_low','_month_pivot_p', '_quarter_pivot_p', '_year_pivot_p',
            '_ma5', '_ma20', '_ma60', '_ma120', '_ma240','_ma480', '_month_fibo_382',
            '_month_fibo_500', '_month_fibo_618','_quarter_fibo_382', '_quarter_fibo_500',
            '_quarter_fibo_618', '_year_fibo_382','_year_fibo_500', '_year_fibo_618']

for name in temp_list:
    filter_stock_df[f'signal{name}'] = filter_stock_df.apply(lambda x: 
        'Tiệm cận kháng cự' if (x[f'from{name}']<0.03) & (x[f'from{name}']>-0.03) & (x[f'count_from{name}']<3) else 
        ('Tiệm cận hỗ trợ' if (x[f'from{name}']<0.03) & (x[f'from{name}']>-0.03) & (x[f'count_from{name}']>=3) else 'Không có tin hiệu'), axis=1)

KeyError: 'count_from_month_high'

In [149]:
filter_stock_df

,stock,date,close,low,high,volume,liquid_ratio,cap,t0_score,t5_score,...,from_quarter_pivot,from_year_fibo_382,from_year_fibo_500,from_year_fibo_618,from_year_high,from_year_low,from_year_ma240,from_year_ma480,from_year_open,from_year_pivot
0,TTH,2024-04-22,4.80,4.40,4.80,2317700.0,3.893724,169.30800,28.328604,7.964513,...,0.035971,0.140088,0.280000,0.459055,-0.157895,0.371429,0.226706,0.454913,0.297297,0.321101
1,AAV,2024-04-22,3.60,3.30,3.60,979500.0,1.641749,246.97575,11.804585,2.550404,...,-0.060870,-0.373128,-0.307692,-0.227003,-0.142857,0.241379,-0.248173,-0.346409,-0.121951,-0.294118
2,BVS,2024-04-22,35.60,33.00,35.60,1482500.0,1.353436,2514.00800,10.307926,-1.617707,...,0.153348,0.164773,0.283264,0.428594,-0.103275,0.447154,0.355284,0.616121,0.396078,0.505884
3,DDG,2024-04-22,3.50,3.20,3.50,974000.0,1.275771,338.92030,8.613196,0.870358,...,-0.339623,-0.875846,-0.850427,-0.811921,-0.435484,0.129032,-0.489144,-0.845826,-0.385965,-0.808394
4,ORS,2024-04-22,14.70,13.90,14.70,4865000.0,1.415759,4942.50000,7.455867,-1.971020,...,-0.126186,0.031435,0.138750,0.270990,-0.209677,0.101124,0.004884,0.188483,-0.007065,0.094217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,POM,2024-04-22,2.96,2.95,3.20,1228600.0,0.641473,1196.90170,-1.653292,-0.765013,...,-0.429306,-0.556074,-0.501264,-0.431012,-0.506667,0.045936,-0.479804,-0.492260,-0.425243,-0.517129
313,SJS,2024-04-22,67.00,67.00,69.00,303700.0,1.749021,8149.93450,-2.606822,-0.916374,...,-0.096223,0.016729,0.101974,0.202821,-0.186893,0.030769,0.106935,0.144073,-0.033189,0.100767
314,QCG,2024-04-22,16.65,16.65,18.80,1556500.0,0.598672,3665.40900,-2.833626,2.371886,...,0.475628,0.279519,0.483296,0.764276,-0.114362,1.018182,0.617107,1.052662,0.632353,0.684654
315,LAS,2024-04-22,18.50,18.40,19.60,2270900.0,1.260015,2384.09100,-3.447021,-6.077217,...,-0.022887,0.068036,0.202697,0.376215,-0.216102,0.321429,0.292312,0.539749,0.321429,0.505527


#### Lưu vào SQL

- Tạo thời gian cập nhật

In [94]:
update_time = pd.DataFrame([f"Cập nhât: {time_series.iloc[0].item().strftime('%d/%m/%Y %H:%M:%S')}"]).rename(columns={0:'date'})

- Lưu các bảng dữ liệu

In [146]:
import socket
from sqlalchemy import create_engine, MetaData

username = 'twan'
password = 'chodom'
database = 't2m'
server = socket.gethostbyname(socket.gethostname())
engine = create_engine(f"mysql+pymysql://{username}:{password}@{server}/{database}")

#Xoá toàn bộ các bảng cũ
conn = engine.connect()
metadata = MetaData()
metadata.reflect(bind=engine)
for table in reversed(metadata.sorted_tables):
    table.drop(engine)
conn.close()

#Lưu các bảng mới
update_time.to_sql(name='update_time', con=engine, if_exists='replace', index=False)
index_card_df.to_sql(name='index_card_df', con=engine, if_exists='replace', index=False)
index_price_chart_df.to_sql(name='index_price_chart_df', con=engine, if_exists='replace', index=False)
ta_index_df.to_sql(name='ta_index_df', con=engine, if_exists='replace', index=False)
nn_td_top_stock.to_sql('nn_td_top_stock', con=engine, if_exists='replace', index=False)
nn_td_20p_df.to_sql('nn_td_20p_df', con=engine, if_exists='replace', index=False)
nn_td_buy_sell_df.to_sql('nn_td_buy_sell_df', con=engine, if_exists='replace', index=False)
market_sentiment.to_sql(name='market_sentiment', con=engine, if_exists='replace', index=False)
itd_score_liquidity_df.to_sql(name='itd_score_liquidity_df', con=engine, if_exists='replace', index=False)
itd_score_liquidity_last.to_sql(name='itd_score_liquidity_last', con=engine, if_exists='replace', index=False)
market_ms.to_sql(name='market_ms', con=engine, if_exists='replace', index=False)
market_top_stock.to_sql(name='market_top_stock', con=engine, if_exists='replace', index=False)
group_score_week.to_sql(name='group_score_week', con=engine, if_exists='replace', index=False)
group_score_month.to_sql(name='group_score_month', con=engine, if_exists='replace', index=False)
eod_score_liquidity_melted.to_sql(name='eod_score_liquidity_melted', con=engine, if_exists='replace', index=False)
market_breath_df.to_sql(name='market_breath_df', con=engine, if_exists='replace', index=False)
group_score_df_5p.to_sql(name='group_score_df_5p', con=engine, if_exists='replace', index=False)
group_score_ranking.to_sql(name='group_score_ranking', con=engine, if_exists='replace', index=False)
eod_group_liquidity_df.to_sql(name='eod_group_liquidity_df', con=engine, if_exists='replace', index=False)
full_industry_ranking.to_sql(name='full_industry_ranking', con=engine, if_exists='replace', index=False)
industry_breath_df.to_sql(name='industry_breath_df', con=engine, if_exists='replace', index=False)
group_score_ranking_melted.to_sql(name='group_score_ranking_melted', con=engine, if_exists='replace', index=False)
itd_score_liquidity_melted.to_sql(name='itd_score_liquidity_melted', con=engine, if_exists='replace', index=False)
eod_score_df.to_sql(name='eod_score_df', con=engine, if_exists='replace', index=False)
group_slicer_df.to_sql(name='group_slicer_df', con=engine, if_exists='replace', index=False)
group_stock_price_index.to_sql(name='group_stock_price_index', con=engine, if_exists='replace', index=False)
group_stock_top_10_df.to_sql(name='group_stock_top_10_df', con=engine, if_exists='replace', index=False)
stock_price_chart_df.to_sql(name='stock_price_chart_df', con=engine, if_exists='replace', index=False)
ta_stock_df.to_sql(name='stock_ta_df', con=engine, if_exists='replace', index=False)
stock_liquidty_score_t0.to_sql(name='stock_liquidty_score_t0', con=engine, if_exists='replace', index=False)
stock_score_week.to_sql(name='stock_score_week', con=engine, if_exists='replace', index=False)
stock_score_month.to_sql(name='stock_score_month', con=engine, if_exists='replace', index=False)
stock_score_power_df.to_sql(name='stock_score_power_df', con=engine, if_exists='replace', index=False)
# filter_stock_df.to_sql(name='filter_stock_df', con=engine, if_exists='replace', index=False)

engine.dispose()

In [147]:
group_score_ranking_melted.columns

Index(['date', 'group_name', 'rank'], dtype='object')

In [96]:
# stock_score_power_df.to_sql(name='stock_score_power_df', con=engine, if_exists='replace', index=False)